In [1]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
import pandas as pd
from matplotlib.patches import FancyArrowPatch
import csv # Import csv module for data saving

BOARD_SIZE = 100

In [2]:
def create_snakes_and_ladders(board_size, num_snakes, num_ladders, max_regeneration_attempts=500): # Added max_regeneration_attempts
    """
    Generates random positions, with regeneration attempt limit.
    """
    attempts = 0 # Initialize attempt counter
    while attempts < max_regeneration_attempts:  # Loop with attempt limit
        attempts += 1
        snakes, ladders = _generate_snakes_and_ladders_once(board_size, num_snakes, num_ladders)

        # Connectivity check (same as before)
        transition_matrix = create_transition_matrix(board_size, snakes, ladders)
        excluded_tiles = set(snakes.keys()) | set(ladders.keys())
        valid_states = [state for state in range(board_size + 1) if state not in excluded_tiles]
        n = len(valid_states)
        Q_check = transition_matrix.iloc[:n, :n].copy()

        has_zero_row = False
        for index, row in Q_check.iterrows():
            if all(row == 0):
                has_zero_row = True
                break

        if not has_zero_row:
            print(f"Valid board generated after {attempts} regeneration attempts.") # Show attempts
            return snakes, ladders
        else:
            print(f"Board layout with isolated states. Regenerating... (Attempt {attempts}/{max_regeneration_attempts})") # Show attempt count

    raise Exception(f"Failed to generate valid board after {max_regeneration_attempts} attempts.") # Raise exception if limit reached
    # OR, alternatively, return None: return None, None # Signal failure to generate board

def _generate_snakes_and_ladders_once(board_size, num_snakes, num_ladders):
    """
    Helper function to generate snakes and ladders with connectivity heuristics.
    """
    snakes = {}
    ladders = {}
    used_tiles = set()

    def generate_endpoints(entity_type, used_tiles):
        attempts = 0
        while attempts < 100:  # Limit attempts
            attempts += 1
            if entity_type == "snake":
                start_tile = random.randint(20, board_size - 1)  # Snakes start further up
            else:  # ladder
                start_tile = random.randint(2, board_size - 30) # Ladders start lower

            if start_tile in used_tiles:
                continue

            if entity_type == "snake":
                # Corrected line: Ensure end_tile range is valid
                end_tile = random.randint(1, max(1, start_tile - 20)) # Snakes go down significantly, ensure valid range
            else:  # ladder
                end_tile = random.randint(start_tile + 10, board_size - 1) # Ladders go up significantly

            if end_tile in used_tiles or end_tile == start_tile:
                continue

            valid_placement = _check_overlap(snakes, ladders, start_tile, end_tile, entity_type) # Use helper

            if valid_placement:
                used_tiles.add(start_tile)
                used_tiles.add(end_tile)
                return start_tile, end_tile
        return None, None # Return None if no valid placement found after attempts


    def _check_overlap(snakes, ladders, start_tile, end_tile, entity_type):
        """Helper function to check for overlaps (same as before)."""
        valid_placement = True
        for s_start, s_end in snakes.items():
            if start_tile == s_start or start_tile == s_end or end_tile == s_start or end_tile == s_end:
                valid_placement = False
                break
        if not valid_placement:
            return False

        for l_start, l_end in ladders.items():
            if start_tile == l_start or start_tile == l_end or end_tile == l_start or end_tile == l_end:
                valid_placement = False
                break
        return valid_placement


    # Generate Snakes - with retry mechanism if placement fails
    snakes_placed = 0
    for _ in range(num_snakes * 2): # Try placing snakes more times than needed
        if snakes_placed >= num_snakes:
            break # Stop if enough snakes placed
        start, end = generate_endpoints("snake", used_tiles)
        if start is not None: # Check if valid placement was found
            snakes[start] = end
            snakes_placed += 1

    # Generate Ladders - with retry mechanism
    ladders_placed = 0
    for _ in range(num_ladders * 2): # Try placing ladders more times than needed
        if ladders_placed >= num_ladders:
            break # Stop if enough ladders placed
        start, end = generate_endpoints("ladder", used_tiles)
        if start is not None: # Check if valid placement was found
            ladders[start] = end
            ladders_placed += 1

    print("\n--- Board Generation (Debugging) ---") # Debugging print
    print("Number of snakes placed:", len(snakes))
    print("Number of ladders placed:", len(ladders))
    print("Snakes:", snakes)
    print("Ladders:", ladders)
    print("--- End Board Generation (Debugging) ---")
    return snakes, ladders



In [3]:
def simulate_game(board_size, snakes, ladders, record_positions=False, record_entity_triggers=False):
    """
    Simulates a single game of Snakes and Ladders.
    
    Args:
        board_size (int): Size of the board
        snakes (dict): Dictionary of snake positions
        ladders (dict): Dictionary of ladder positions
        record_positions (bool): Whether to record all positions visited
        record_entity_triggers (bool): Whether to record snake/ladder triggers
        
    Returns:
        tuple: (turns taken, positions visited, entity triggers)
    """
    position = 0
    turns = 0
    positions_visited = [] if record_positions else None
    entity_triggers = [] if record_entity_triggers else None

    if record_positions:
        positions_visited.append(position)

    while position < board_size:
        roll = random.randint(1, 6)
        new_position = position + roll

        if new_position > board_size:
            new_position = board_size - (new_position - board_size)

        position = new_position
        turns += 1

        if record_positions:
            positions_visited.append(position)

        if position in snakes:
            if record_entity_triggers:
                entity_triggers.append(position)
            position = snakes[position]
            if record_positions:
                positions_visited.append(position)
        elif position in ladders:
            if record_entity_triggers:
                entity_triggers.append(position)
            position = ladders[position]
            if record_positions:
                positions_visited.append(position)

    return turns, positions_visited, entity_triggers

In [4]:
def create_transition_matrix(board_size, snakes, ladders):
    """
    Creates a transition matrix for the Snakes and Ladders Markov model,
    EXCLUDING snake heads and ladder bases as states.

    Args:
        board_size (int): Size of the board
        snakes (dict): Dictionary of snake positions (head: tail)
        ladders (dict): Dictionary of ladder positions (base: top)

    Returns:
        pandas.DataFrame: Transition matrix (DataFrame)
    """
    # Identify snake heads and ladder bases to exclude as states
    excluded_tiles = set(snakes.keys()) | set(ladders.keys())
    valid_states = [state for state in range(board_size + 1) if state not in excluded_tiles]
    num_states = len(valid_states)

    transition_matrix = pd.DataFrame(0.0, index=valid_states, columns=valid_states)

    state_to_index = {state: index for index, state in enumerate(valid_states)} # Map state to index

    for current_tile in valid_states:
        if current_tile == board_size: # Absorbing state
            transition_matrix.loc[current_tile, current_tile] = 1.0
            continue

        probabilities = [0.0] * num_states # Initialize probabilities for valid states

        for roll in range(1, 7):
            next_position_tentative = current_tile + roll

            if next_position_tentative > board_size:
                next_position_tentative = board_size - (next_position_tentative - board_size) # Bounce back

            next_position = next_position_tentative # Initialize assuming no snake/ladder

            # Check for snakes and ladders at the *tentative* next position
            if next_position_tentative in snakes:
                next_position = snakes[next_position_tentative] # Move to snake tail
            elif next_position_tentative in ladders:
                next_position = ladders[next_position_tentative] # Move to ladder top

            # Ensure next_position is a valid state (not a snake head or ladder base)
            while next_position in excluded_tiles:
                if next_position in snakes:
                    next_position = snakes[next_position] # Chain snake
                elif next_position in ladders:
                    next_position = ladders[next_position] # Chain ladder
                else: # Should not happen in a well-formed board, but as a safeguard
                    break # Break out if stuck in an infinite loop

            if next_position in valid_states:
                next_state_index = state_to_index[next_position] # Get index in reduced state space
                probabilities[next_state_index] += 1/6.0
            elif next_position == board_size: # Handle reaching the goal (absorbing state)
                next_state_index = state_to_index[board_size]
                probabilities[next_state_index] += 1/6.0


        # Assign probabilities to the transition matrix row
        for next_state_index, prob in enumerate(probabilities):
            next_state = valid_states[next_state_index]
            transition_matrix.loc[current_tile, next_state] = prob


    return transition_matrix

def calculate_expected_turns_markov(transition_matrix, board_size, snakes, ladders):
    # Get valid states (excluding snake heads/ladder bases AND the absorbing state)
    excluded_tiles = set(snakes.keys()) | set(ladders.keys()) | {board_size}
    valid_states = [state for state in range(board_size) if state not in excluded_tiles]
    n = len(valid_states)
    
    # Extract Q matrix (only transient states)
    Q = transition_matrix.loc[valid_states, valid_states].copy()
    I = pd.DataFrame(np.identity(n), index=valid_states, columns=valid_states)
    
    try:
        N = np.linalg.inv(I - Q)
        ones = np.ones(n)
        expected_turns = N.dot(ones)
        return expected_turns[0]  # Return the expected turns from state 0
    except np.linalg.LinAlgError as e:
        print("Matrix inversion failed. Check transition matrix structure.")
        raise e
    
def calculate_steady_state_distribution(transition_matrix, board_size, snakes, ladders, initial_state=0, num_iterations=500): # Added snakes, ladders
    """
    Calculates the steady-state distribution of tile occupation probabilities,
    ADJUSTED FOR REDUCED STATE SPACE.
    """
    # Get valid states - IMPORTANT: Re-calculate valid states to be consistent with transition_matrix
    excluded_tiles = set(snakes.keys()) | set(ladders.keys())
    valid_states = [state for state in range(board_size + 1) if state not in excluded_tiles]
    num_states = len(valid_states) # Recalculate num_states based on valid_states

    state_distribution = pd.Series([0.0] * num_states, index=valid_states) # Use valid_states as index
    state_distribution[initial_state] = 1.0

    # Ensure state_distribution index matches transition_matrix columns - EXPLICIT CHECK
    state_distribution.index = transition_matrix.columns # Force index alignment (although now it should match already)

    # Debugging Prints:
    print("\n--- Debugging Information (calculate_steady_state_distribution) ---")
    print("Shape of state_distribution (should be num_states):", state_distribution.shape)
    print("Shape of transition_matrix (should be num_states x num_states):", transition_matrix.shape)
    print("\nstate_distribution Index (first 10):", state_distribution.index[:10]) # Print first 10 indices
    print("\ntransition_matrix Columns Index (first 10):", transition_matrix.columns[:10]) # Print first 10 column indices
    print("\nstate_distribution dtype:", state_distribution.dtype)
    print("\ntransition_matrix dtypes (first 5 columns):", transition_matrix.dtypes.head())
    print("--- End Debugging Information ---")


    for _ in range(num_iterations):
        # state_distribution = state_distribution.dot(transition_matrix) # Original dot product
        state_distribution = state_distribution @ transition_matrix # Try @ operator - ALTERNATIVE DOT PRODUCT

    return state_distribution

def simulate_game_markov_distribution(transition_matrix, num_games, board_size):
    """
    Simulates games directly from the Markov transition matrix to generate turn distribution.

    Args:
        transition_matrix (pd.DataFrame): Markov transition matrix.
        num_games (int): Number of games to simulate.
        board_size (int): Size of the board.

    Returns:
        list: List of turn counts for simulated games.
    """

    turn_counts_markov = []
    start_state_index = 0 # Assuming starting from state 0

    for _ in range(num_games):
        current_state = start_state_index
        turns = 0
        while current_state < board_size: # While not in absorbing state (state 100)
            probabilities = transition_matrix.loc[current_state].values # Get transition probabilities for current state
            next_state = random.choices(transition_matrix.columns, weights=probabilities, k=1)[0] # Probabilistic transition
            current_state = next_state
            turns += 1
        turn_counts_markov.append(turns)
    return turn_counts_markov

In [9]:
def create_labeled_board_representation(board_size, snakes, ladders, board_linear_size):
    """
    Creates a NumPy array representation of a Snakes and Ladders board with labels,
    dynamically sized based on board_linear_size.

    Args:
        board_size (int): Total number of tiles on the board
        snakes (dict): Dictionary of snake positions
        ladders (dict): Dictionary of ladder positions
        board_linear_size (int): Linear dimension of the square board (e.g., 10 for 10x10)

    Returns:
        numpy.ndarray: NumPy array of strings representing tile labels, sized board_linear_size x board_linear_size
    """
    grid_size = board_linear_size
    grid = np.empty((grid_size, grid_size), dtype=object)  # NumPy array of objects, dynamic size

    for tile in range(1, board_size + 1): # Iterate only up to board_size
        row = (tile - 1) // grid_size # Use grid_size here
        col = (tile - 1) % grid_size # Use grid_size here
        if row % 2 == 1:
            col = (grid_size - 1) - col # Use grid_size here

        tile_label = str(tile)

        if tile in snakes:
            tile_label = f'S{tile}→{snakes[tile]}'
        elif tile in ladders:
            tile_label = f'L{tile}→{ladders[tile]}'

        grid[grid_size - 1 - row, col] = tile_label # Use grid_size for row index as well (and invert row)


    return grid


def plot_board_heatmap_representation(board_size, snakes, ladders):
    """
    Plots a Snakes and Ladders board representation using sns.heatmap with labels.

    Args:
        board_size (int): Size of the board
        snakes (dict): Dictionary of snake positions
        ladders (dict): Dictionary of ladder positions
    """
    grid_representation = create_labeled_board_representation(board_size, snakes, ladders)
    empty_board_data = np.zeros((10, 10)) # Dummy data for heatmap

    fig, ax = plt.subplots(figsize=(12, 10))
    sns.heatmap(empty_board_data, annot=grid_representation, fmt='s', cmap='Greys', # Use 'Wistia' or 'Greys' cmap
                linewidths=1, linecolor='black', cbar=False, annot_kws={"fontsize": 9, "fontweight": "bold"}, ax=ax) # Adjust fontsize here

    ax.set_title("Snakes and Ladders Board", fontsize=14)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.invert_yaxis() # Invert y-axis to match board layout

    return plt # Return plt object

def plot_turn_distribution(turn_counts_simulation, turn_counts_markov, num_snakes, num_ladders, num_simulations):
    """
    Plots and compares turn distributions from simulations and Markov model.
    """
    plt.figure(figsize=(10, 6))
    
    # Determine bin size
    max_turns = max(max(turn_counts_simulation), max(turn_counts_markov))
    min_turns = min(min(turn_counts_simulation), min(turn_counts_markov))

    bin_size = max(1, (max_turns - min_turns) // 20)

    # Plot histograms for both simulation and Markov data
    plt.hist(turn_counts_simulation, bins=range(min_turns, max_turns + bin_size, bin_size),
             align='left', rwidth=0.8, color='skyblue', edgecolor='black', density=True, label='Simulation')
    plt.hist(turn_counts_markov, bins=range(min_turns, max_turns + bin_size, bin_size),
             align='left', rwidth=0.8, color='coral', edgecolor='black', density=True, alpha=0.7, label='Markov Model') # Use different color, reduce alpha

    plt.xlabel("Number of Turns")
    plt.ylabel("Probability")
    plt.title(f"Comparison of Turn Distributions\n(Simulation vs. Markov Model) - ({num_snakes} Snakes, {num_ladders} Ladders, {num_simulations} Simulations)")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.xticks(range(min_turns, max_turns + 1, bin_size*2)) # Adjusted x-ticks for better readability
    plt.legend() # Show legend to distinguish datasets
    plt.tight_layout()
    plt.show()



def plot_board_heatmap(all_positions_visited, board_size, snakes, ladders, num_simulations):
    """
    Creates a heatmap showing frequency of visits to each board position.
    """
    
    tile_counts = Counter()
    for positions_list in all_positions_visited:
        tile_counts.update(positions_list)

    board_grid = np.zeros((10, 10))
    for tile in range(1, board_size + 1):
        row = (tile - 1) // 10  
        col = (tile - 1) % 10
        if row % 2 == 1:
            col = 9 - col
        board_grid[9 - row, col] = tile_counts.get(tile, 0)

    plt.figure(figsize=(12, 10))
    sns.heatmap(board_grid, annot=False, fmt="d", cmap="viridis",
                linewidths=.5, cbar_kws={'label': 'Visit Frequency'})
    
    # Add snake and ladder markers
    for start, end in snakes.items():
        row = (start - 1) // 10
        col = (start - 1) % 10 if row % 2 == 0 else 9 - ((start - 1) % 10)
        plt.text(col, 9 - row, 'S', color='white', ha='center', va='center', fontsize=8)
    
    for start, end in ladders.items():
        row = (start - 1) // 10
        col = (start - 1) % 10 if row % 2 == 0 else 9 - ((start - 1) % 10)
        plt.text(col, 9 - row, 'L', color='white', ha='center', va='center', fontsize=8)

    plt.title(f"Visit Frequency Heatmap\n({num_simulations} Simulations)")
    plt.xticks(np.arange(0, 10, 1), labels=[f'{i*10+1}-{(i+1)*10}' for i in range(10)])
    plt.yticks([])
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
def plot_entity_trigger_heatmap(all_entity_triggers, board_size, snakes, ladders, num_simulations):
    """
    Creates a heatmap showing frequency of snake and ladder triggers.
    Fixed to show correct start→end labels for snakes and ladders.
    """
    tile_counts = Counter()
    for triggers in all_entity_triggers:
        tile_counts.update(triggers)

    board_grid = np.zeros((10, 10))
    
    # Fill the grid with trigger counts
    for tile in range(1, board_size + 1):
        row = (tile - 1) // 10
        col = (tile - 1) % 10
        if row % 2 == 1:
            col = 9 - col
        board_grid[9 - row, col] = tile_counts.get(tile, 0)

    plt.figure(figsize=(12, 10))
    
    # Create heatmap with white for zero values
    sns.heatmap(board_grid, annot=True, fmt=".0f", cmap="YlOrRd",
                linewidths=.5, cbar_kws={'label': 'Trigger Frequency'},
                mask=(board_grid == 0))
    
    # Add a background color for cells with zero values
    plt.gca().patch.set_facecolor('lightgray')
    
    # Add markers for snakes and ladders with correct start→end format
    for start, end in snakes.items():
        row = (start - 1) // 10
        col = (start - 1) % 10 if row % 2 == 0 else 9 - ((start - 1) % 10)
        plt.text(col, 9 - row, f'S{start}→{end}', color='darkblue', 
                ha='center', va='center', fontsize=8, fontweight='bold')
    
    for start, end in ladders.items():
        row = (start - 1) // 10
        col = (start - 1) % 10 if row % 2 == 0 else 9 - ((start - 1) % 10)
        plt.text(col, 9 - row, f'L{start}→{end}', color='darkgreen', 
                ha='center', va='center', fontsize=8, fontweight='bold')

    plt.title(f"Snake and Ladder Trigger Frequency\n({num_simulations} Simulations)")
    plt.xticks([])
    plt.yticks([])
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

def plot_steady_state_heatmap(steady_state_distribution, board_size, snakes, ladders): # Added snakes, ladders
    """
    Generates and displays a heatmap of steady-state tile occupation probabilities,
    ADJUSTED FOR REDUCED STATE SPACE.
    """
    plt.figure(figsize=(12, 10))
    board_grid_steady_state = np.zeros((10, 10))

    # Get valid states - IMPORTANT: Use valid states from transition_matrix creation
    excluded_tiles = set(snakes.keys()) | set(ladders.keys())
    valid_states = [state for state in range(board_size + 1) if state not in excluded_tiles]

    for tile in valid_states: # Iterate over valid_states only - CORRECTED LOOP
        if tile > 0 and tile <= board_size: # Ensure tile is within board range and not state 0
            row = (tile - 1) // 10
            col = (tile - 1) % 10
            if row % 2 == 1:
                col = 9 - col
            board_grid_steady_state[9 - row, col] = steady_state_distribution[tile] # Access using 'tile' index


    sns.heatmap(board_grid_steady_state, cmap="viridis", linewidths=.5, cbar_kws={'label': 'Steady-State Probability'})
    plt.title(f"Steady-State Tile Occupation Probability Heatmap")
    plt.xticks([])
    plt.yticks([])
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()
    
def plot_win_prob_vs_ratio_separated_board_sizes(results_ns_fixed, results_nl_fixed, board_sizes, save_folder="plots/ratio_analysis", filename_prefix="win_prob_vs_ratio_board_sizes"):
    """
    Plots Win Probability vs Ns/Nl Ratio, with multiple lines for different board sizes on a single plot.
    Separated for Ns-fixed and Nl-fixed scenarios. Saves plot and data to CSV.
    """
    df_ns_fixed = pd.DataFrame(results_ns_fixed)
    df_nl_fixed = pd.DataFrame(results_nl_fixed)

    win_fractions = [0.5, 1/3, 0.25]
    fraction_labels = {'0.5': 'N/2', '0.3333333333333333': 'N/3', '0.25': 'N/4'}
    line_styles = ['-', '--', ':'] # Line styles for N/2, N/3, N/4 probs
    markers = ['o', '^', 's'] # Markers for N/2, N/3, N/4 probs
    colours = ['blue', 'purple', 'teal'] # Colours for N/2, N/3, N/4 probs


    # --- Plot for Ns Fixed ---
    fig = plt.figure(figsize=(12, 6))
    ax_ns = fig.add_subplot(1, 1, 1) # Single subplot


    for size in sorted(list(set(df_ns_fixed['board_size']))): # Iterate through board sizes
        df_subset_ns = df_ns_fixed[df_ns_fixed['board_size'] == size]
        for i, fraction in enumerate(win_fractions): # Iterate through win fractions
            prob_col_name = f'prob_win_{fraction_labels[str(fraction)]}_sim'.lower().replace('/', '').replace('.', '')
            ax_ns.plot(df_subset_ns['ratio'], df_subset_ns[prob_col_name], 
                       marker=markers[i], linestyle=line_styles[i], color=colours[i], 
                       label=f'{size}x{size} - N/{fraction_labels[str(fraction)]} Turns') # Label with board size and win condition


    ax_ns.set_xlabel("Ns/Nl Ratio")
    ax_ns.set_ylabel("Probability of Winning within N Turns")
    ax_ns.set_title("Win Probability vs Ns/Nl Ratio (Ns Fixed, by Board Size)")
    ax_ns.legend(loc='upper right', fontsize='small', ncol=2) # Adjusted legend
    ax_ns.grid(True)
    ax_ns.set_ylim(bottom=0, top=1.05) # Set y-axis limits
    fig.tight_layout()


    if save_folder:
        ns_fixed_folder = os.path.join(save_folder, "ns_fixed")
        os.makedirs(ns_fixed_folder, exist_ok=True)
        filepath_png = os.path.join(ns_fixed_folder, filename_prefix + "_ns_fixed.png")
        fig.savefig(filepath_png, bbox_inches='tight')
        print(f"Win prob vs ratio (Ns fixed, by board size) plot saved to: {filepath_png}")
    else:
        plt.show()
    plt.close(fig)



    # --- Plot for Nl Fixed ---
    fig = plt.figure(figsize=(12, 6))
    ax_nl = fig.add_subplot(1, 1, 1) # Single subplot


    for size in sorted(list(set(df_nl_fixed['board_size']))): # Iterate through board sizes
        df_subset_nl = df_nl_fixed[df_nl_fixed['board_size'] == size]
        for i, fraction in enumerate(win_fractions): # Iterate through win fractions
            prob_col_name = f'prob_win_{fraction_labels[str(fraction)]}_sim'.lower().replace('/', '').replace('.', '')
            ax_nl.plot(df_subset_nl['ratio'], df_subset_nl[prob_col_name], 
                       marker=markers[i], linestyle=line_styles[i], color=colours[i],
                       label=f'{size}x{size} - N/{fraction_labels[str(fraction)]} Turns') # Label with board size and win condition


    ax_nl.set_xlabel("Ns/Nl Ratio")
    ax_nl.set_ylabel("Probability of Winning within N Turns")
    ax_nl.set_title("Win Probability vs Ns/Nl Ratio (Nl Fixed, by Board Size)")
    ax_nl.legend(loc='upper right', fontsize='small', ncol=2) # Adjusted legend
    ax_nl.grid(True)
    ax_nl.set_ylim(bottom=0, top=1.05) # Set y-axis limits
    fig.tight_layout()


    if save_folder:
        nl_fixed_folder = os.path.join(save_folder, "nl_fixed")
        os.makedirs(nl_fixed_folder, exist_ok=True)
        filepath_png = os.path.join(nl_fixed_folder, filename_prefix + "_nl_fixed.png")
        fig.savefig(filepath_png, bbox_inches='tight')
        print(f"Win prob vs ratio (Nl fixed, by board size) plot saved to: {filepath_png}")
    else:
        plt.show()
    plt.close(fig)

In [8]:
def plot_board_heatmap_representation_with_arrows(board_size, snakes, ladders, board_linear_size):
    """
    Plots a Snakes and Ladders board representation using sns.heatmap with labels AND arrows,
    dynamically sized.

    Args:
        board_size (int): Size of the board
        snakes (dict): Dictionary of snake positions
        ladders (dict): Dictionary of ladder positions
        board_linear_size (int): Linear dimension of the square board

    """
    cmap = sns.color_palette(['lightgray', 'red', 'green']) # Colour map

    grid_representation = create_labeled_board_representation(board_size, snakes, ladders, board_linear_size)
    grid_size = board_linear_size # Get grid size
    empty_board_data = np.zeros((grid_size, grid_size)) # Dynamic size for heatmap data

    fig, ax = plt.subplots(figsize=(12, 10))
    sns.heatmap(empty_board_data, annot=grid_representation, fmt='s', cmap=cmap,
                linewidths=1, linecolor='black', cbar=False, annot_kws={"fontsize": 8, "fontweight": "bold"}, ax=ax)

    ax.set_title(f"Snakes and Ladders Board ({board_size} Tiles) with Arrows - {board_linear_size}x{board_linear_size}", fontsize=14) # More descriptive title
    ax.set_xticks([])
    ax.set_yticks([])
    ax.invert_yaxis()

    # --- Add Arrows ---
    arrow_props_snake=dict(arrowstyle='-|>', mutation_scale=15, lw=1.5, color='red', connectionstyle="arc3,rad=0.") # Straight red arrows
    arrow_props_ladder=dict(arrowstyle='-|>', mutation_scale=15, lw=1.5, color='green', connectionstyle="arc3,rad=0.") # Straight green arrows


    for start_tile, end_tile in snakes.items():
        start_pos = _tile_to_position_dynamic_grid(start_tile, board_linear_size) # Use dynamic position function
        end_pos = _tile_to_position_dynamic_grid(end_tile, board_linear_size)     # Use dynamic position function
        arrow = FancyArrowPatch(start_pos, end_pos, transform=ax.transData, **arrow_props_snake,)
        ax.add_patch(arrow)

    for start_tile, end_tile in ladders.items():
        start_pos = _tile_to_position_dynamic_grid(start_tile, board_linear_size) # Use dynamic position function
        end_pos = _tile_to_position_dynamic_grid(end_tile, board_linear_size)     # Use dynamic position function
        arrow = FancyArrowPatch(start_pos, end_pos, transform=ax.transData, **arrow_props_ladder)
        ax.add_patch(arrow)
    # --- End Arrows ---

    return plt # Return plt object


def _tile_to_position_dynamic_grid(tile, board_linear_size):
    """Helper function to convert tile number to heatmap coordinates for dynamic grid."""
    grid_size = board_linear_size
    row = (tile - 1) // grid_size
    col = (tile - 1) % grid_size
    if row % 2 == 1:
        col = (grid_size - 1) - col
    return [col + 0.5, grid_size - 1 - row + 0.5] # Adjusted for dynamic grid size and inverted row

In [7]:
def run_simulations_and_markov(board_size, snakes, ladders, num_simulations, num_markov_simulations):
    """
    Encapsulates the simulation and Markov analysis to reduce repetition in the main loop.
    """
    turn_counts_simulation = []
    win_counts_n_turns_sim = {0.5: 0, 1/3: 0, 0.25: 0} # N/2, N/3, N/4 counters

    for _ in range(num_simulations):
        turns, _, _ = simulate_game(board_size, snakes, ladders)
        turn_counts_simulation.append(turns)
        for fraction, count in win_counts_n_turns_sim.items(): # Check for each fraction
            if turns <= board_size * fraction:
                win_counts_n_turns_sim[fraction] += 1


    avg_turns_simulation = sum(turn_counts_simulation) / num_simulations
    win_prob_n_turns_sim = {fraction: count / num_simulations for fraction, count in win_counts_n_turns_sim.items()} # Probabilities

    transition_matrix = create_transition_matrix(board_size, snakes, ladders)
    try:
        expected_turns_markov = calculate_expected_turns_markov(transition_matrix, board_size, snakes, ladders)
    except np.linalg.LinAlgError:
        expected_turns_markov = np.nan # Use NaN if Markov fails

    win_prob_n_turns_sim = {fraction: count / num_simulations for fraction, count in win_counts_n_turns_sim.items()} # Probabilities

    print("\n--- Win Probabilities (Debugging) ---") # Debugging print
    print("Win Probabilities (N/2 turns):", win_prob_n_turns_sim[0.5])
    print("Win Probabilities (N/3 turns):", win_prob_n_turns_sim[1/3])
    print("Win Probabilities (N/4 turns):", win_prob_n_turns_sim[0.25])
    print("--- End Win Probabilities (Debugging) ---")

    transition_matrix = create_transition_matrix(board_size, snakes, ladders)
    
    return {
        'avg_turns_simulation': avg_turns_simulation,
        'expected_turns_markov': expected_turns_markov,
        'win_prob_n_turns_sim': win_prob_n_turns_sim,
    }


In [6]:
def plot_avg_time_vs_ratio_individual(results, board_size, fixed_entity, save_folder="plots/ratio_analysis_individual"):
    """
    Plots Average Game Time vs Ns/Nl Ratio for a SINGLE board size and fixed entity type.
    Generates individual plots (not subplots) for better clarity.

    Args:
        results (dict):  Either results_ns_fixed or results_nl_fixed dictionary.
        board_size (int): The linear board size (e.g., 8, 10, 12...).
        fixed_entity (str): Either "Ns" or "Nl", indicating which entity is fixed.
        save_folder (str): Base folder to save plots.
    """

    df = pd.DataFrame(results)
    df_subset = df[df['board_size'] == board_size]

    fig, ax = plt.subplots(figsize=(8, 6)) # Create a new figure and axes for EACH plot

    # Plot Simulation data
    ax.plot(df_subset['ratio'], df_subset['avg_time_sim'], marker='o', linestyle='-', label='Simulation', color='blue' if fixed_entity == "Ns" else 'green', linewidth=2, markersize=8)

    # Plot Markov Model data
    ax.plot(df_subset['ratio'], df_subset['avg_time_markov'], marker='x', linestyle='--', label='Markov', color='blue' if fixed_entity == "Ns" else 'green', linewidth=2, markersize=8)

    ax.set_xlabel("Ns/Nl Ratio")
    ax.set_ylabel("Average Game Time (Turns)")
    ax.set_title(f"Avg Game Time vs Ns/Nl Ratio ({fixed_entity} Fixed, Board Size: {board_size}x{board_size})") # Clear title
    ax.legend(loc='best', fontsize='small')
    ax.grid(True)

    # --- Automatic Y-axis Scaling ---
    # Get the maximum and minimum data values for this subset.
    max_avg_time = df_subset[['avg_time_sim', 'avg_time_markov']].max().max()
    min_avg_time = df_subset[['avg_time_sim', 'avg_time_markov']].min().min()
    
    # Set y-axis limits with some padding:
    padding = (max_avg_time - min_avg_time) * 0.1  # 10% padding
    ax.set_ylim(bottom=max(0, min_avg_time - padding), top=max_avg_time + padding)

    fig.tight_layout()


    if save_folder:
        entity_folder = os.path.join(save_folder, f"{fixed_entity.lower()}_fixed") # ns_fixed or nl_fixed
        size_folder = os.path.join(entity_folder, f"board_{board_size}x{board_size}")
        os.makedirs(size_folder, exist_ok=True)
        filepath_png = os.path.join(size_folder, f"avg_time_vs_ratio_{board_size}x{board_size}_{fixed_entity}_fixed.png")
        fig.savefig(filepath_png, bbox_inches='tight')
        print(f"Average game time vs ratio plot ({board_size}x{board_size}, {fixed_entity} Fixed) saved to: {filepath_png}")

        # --- Save raw data to CSV (Optional - for individual plots) ---
        # You *could* also save the data for this specific plot to a CSV, if you want
        # very granular data files.  Uncomment the following if needed.
        # filepath_csv = os.path.join(size_folder, f"avg_time_vs_ratio_{board_size}x{board_size}_{fixed_entity}_fixed.csv")
        # df_subset.to_csv(filepath_csv, index=False)
        # print(f"Data saved to: {filepath_csv}")


    else:
        plt.show()

    plt.close(fig)

In [4]:
def run_analysis(board_size=100, num_snakes=10, num_ladders=10, num_simulations=10000, num_markov_simulations=10000, display_labeled_board=False, board_linear_size=10, save_plots=True): # Added save_plots flag
    """
    Runs the complete analysis of the Snakes and Ladders game with optional labeled board display, saving plots.
    """

    turn_counts_simulation = []
    all_positions_visited = []
    all_entity_triggers = []

    snakes, ladders = create_snakes_and_ladders(board_size, num_snakes, num_ladders)

    print("\nBoard Configuration:")
    print("Snakes:", {k: v for k, v in snakes.items()})
    print("Ladders:", {k: v for k, v in ladders.items()})

    if display_labeled_board:
        save_board_folder = None # Don't save individual board layout if just displaying
        if save_plots:
            save_board_folder = "plots/board_layouts" # Save board layouts in this folder if saving is enabled
        plot_board_heatmap_representation_with_arrows(board_size, snakes, ladders, board_linear_size, save_folder=save_board_folder, filename=f"board_layout_{board_size}tiles_{num_snakes}s_{num_ladders}ladders.png") # Save board layout

    for _ in range(num_simulations):
        turns, positions, triggers = simulate_game(board_size, snakes, ladders,
                                                 record_positions=True,
                                                 record_entity_triggers=True)
        turn_counts_simulation.append(turns)
        all_positions_visited.append(positions)
        all_entity_triggers.append(triggers)

    avg_turns_simulation = sum(turn_counts_simulation) / num_simulations
    print(f"\nSimulation Results:")
    print(f"Average Turns (Simulation): {avg_turns_simulation:.2f}")
    print(f"Min Turns: {min(turn_counts_simulation)}")
    print(f"Max Turns: {max(turn_counts_simulation)}")

    transition_matrix = create_transition_matrix(board_size, snakes, ladders)
    try:
        expected_turns_markov = calculate_expected_turns_markov(transition_matrix, board_size, snakes, ladders)
        print(f"\nMarkov Analysis:")
        print(f"Expected Turns (Markov Model): {expected_turns_markov:.2f}")
    except np.linalg.LinAlgError:
        print("Markov analysis failed due to singular matrix")
        expected_turns_markov = None

    steady_state_distribution = calculate_steady_state_distribution(transition_matrix, board_size, snakes, ladders)

    # Simulate games from Markov model distribution
    turn_counts_markov = simulate_game_markov_distribution(transition_matrix, num_markov_simulations, board_size)

    if save_plots: # Save plots if save_plots is True
        base_folder = "plots/single_board_analysis" # Base folder for single board analysis
        board_config_folder = os.path.join(base_folder, f"board_{board_size}tiles_{num_snakes}s_{num_ladders}ladders") # Subfolder for board config
        plot_turn_distribution(turn_counts_simulation, turn_counts_markov, num_snakes, num_ladders, num_simulations, save_folder=board_config_folder, filename="turn_distribution.png")
        plot_board_heatmap(all_positions_visited, board_size, snakes, ladders, num_simulations, save_folder=board_config_folder, filename="visit_heatmap.png")
        plot_entity_trigger_heatmap(all_entity_triggers, board_size, snakes, ladders, num_simulations, save_folder=board_config_folder, filename="entity_trigger_heatmap.png")
        plot_steady_state_heatmap(steady_state_distribution, board_size, snakes, ladders, save_folder=board_config_folder, filename="steady_state_heatmap.png")
    else: # Show plots if not saving
        plot_turn_distribution(turn_counts_simulation, turn_counts_markov, num_snakes, num_ladders, num_simulations)
        plot_board_heatmap(all_positions_visited, board_size, snakes, ladders, num_simulations)
        plot_entity_trigger_heatmap(all_entity_triggers, board_size, snakes, ladders, num_simulations)
        plot_steady_state_heatmap(steady_state_distribution, board_size, snakes, ladders)



def run_board_scaling_analysis(board_sizes, entity_density=0.1, num_ratios=4, num_simulations=1000, num_markov_simulations=1000, display_board=False, save_plots=True):
    """
    Runs analysis, calls individual and faceted plotting functions.
    """

    ratio_increments = np.linspace(0.5, 2.0, num_ratios)

    # Data storage (remains same)
    results_ns_fixed = {'board_size': [], 'ratio': [], 'avg_time_sim': [], 'avg_time_markov': [],
                            'prob_win_n2_sim': [], 'prob_win_n3_sim': [], 'prob_win_n4_sim': []}
    results_nl_fixed = {'board_size': [], 'ratio': [], 'avg_time_sim': [], 'avg_time_markov': [],
                            'prob_win_n2_sim': [], 'prob_win_n3_sim': [], 'prob_win_n4_sim': []}


    for board_size_linear in board_sizes:
      # ... (rest of the board size and ratio loop code - no changes within the loops) ...
        # --- Operation Set 1: Ns Fixed by Density, Nl by Ratio ---
        board_area = board_size_linear**2 # board_area is defined here
        board_size = board_area # Use board_area for board_size
        num_snakes_fixed = int(entity_density * board_area)
        print(f"\n--- Board Size: {board_size_linear}x{board_size_linear} ({board_area} tiles), Ns Fixed = {num_snakes_fixed} ---")

        for ratio in ratio_increments:
            num_ladders = max(1, int(num_snakes_fixed / ratio))
            print(f"\n  Ratio (Ns/Nl): {ratio:.1f}, Nl: {num_ladders}")

            snakes, ladders = create_snakes_and_ladders(board_size, num_snakes_fixed, num_ladders)

            if display_board:
                save_board_folder = None
                if save_plots:
                    save_board_folder = os.path.join("plots/board_layouts", "board_scaling", "ns_fixed", f"board_{board_size_linear}x{board_size_linear}", f"ratio_{ratio:.1f}")
                board_plot = plot_board_heatmap_representation_with_arrows(board_size, snakes, ladders, board_size_linear, save_folder=save_board_folder, filename="board_layout.png")
                if board_plot:
                    board_plot.suptitle(f"Ns Fixed, Ratio {ratio:.1f}, Board {board_size_linear}x{board_size_linear}")

            sim_results = run_simulations_and_markov(board_size, snakes, ladders, num_simulations, num_markov_simulations)

            # Store results
            results_ns_fixed['board_size'].append(board_size_linear)
            results_ns_fixed['ratio'].append(ratio)
            results_ns_fixed['avg_time_sim'].append(sim_results['avg_turns_simulation'])
            results_ns_fixed['avg_time_markov'].append(sim_results['expected_turns_markov'])
            results_ns_fixed['prob_win_n2_sim'].append(sim_results['win_prob_n_turns_sim'][0.5])
            results_ns_fixed['prob_win_n3_sim'].append(sim_results['win_prob_n_turns_sim'][1/3])
            results_ns_fixed['prob_win_n4_sim'].append(sim_results['win_prob_n_turns_sim'][0.25])


        # --- Operation Set 2: Nl Fixed by Density, Ns by Ratio ---
        num_ladders_fixed = int(entity_density * board_area)
        print(f"\n--- Board Size: {board_size_linear}x{board_size_linear} ({board_area} tiles), Nl Fixed = {num_ladders_fixed} ---")

        for ratio in ratio_increments:
            num_snakes = max(1, int(num_ladders_fixed * ratio))
            print(f"\n  Ratio (Ns/Nl): {ratio:.1f}, Ns: {num_snakes}")

            snakes, ladders = create_snakes_and_ladders(board_size, num_snakes, num_ladders_fixed)

            if display_board:
                save_board_folder = None
                if save_plots:
                    save_board_folder = os.path.join("plots/board_layouts", "board_scaling", "nl_fixed", f"board_{board_size_linear}x{board_size_linear}", f"ratio_{ratio:.1f}")
                board_plot = plot_board_heatmap_representation_with_arrows(board_size, snakes, ladders, board_size_linear, save_folder=save_board_folder, filename="board_layout.png")
                if board_plot:
                    board_plot.suptitle(f"Nl Fixed, Ratio {ratio:.1f}, Board {board_size_linear}x{board_size_linear}")

            sim_results = run_simulations_and_markov(board_size, snakes, ladders, num_simulations, num_markov_simulations)

            # Store results
            results_nl_fixed['board_size'].append(board_size_linear)
            results_nl_fixed['ratio'].append(ratio)
            results_nl_fixed['avg_time_sim'].append(sim_results['avg_turns_simulation'])
            results_nl_fixed['avg_time_markov'].append(sim_results['expected_turns_markov'])
            results_nl_fixed['prob_win_n2_sim'].append(sim_results['win_prob_n_turns_sim'][0.5])
            results_nl_fixed['prob_win_n3_sim'].append(sim_results['win_prob_n_turns_sim'][1/3])
            results_nl_fixed['prob_win_n4_sim'].append(sim_results['win_prob_n_turns_sim'][0.25])


    # --- Create DataFrames AFTER the loops have completed ---
    df_ns_fixed = pd.DataFrame(results_ns_fixed)
    df_nl_fixed = pd.DataFrame(results_nl_fixed)

    # --- Create Plots with Saving ---
    if save_plots:
        plot_folder_board_size = os.path.join("plots", "board_scaling")
        plot_avg_time_vs_board_size_separated(results_ns_fixed, results_nl_fixed, save_folder=plot_folder_board_size, filename_prefix="avg_time_vs_board_size")
        plot_win_prob_vs_board_size_separated(results_ns_fixed, results_nl_fixed, save_folder=plot_folder_board_size, filename_prefix="win_prob_vs_board_size")

        # Individual ratio plots:
        plot_folder_ratio_individual = os.path.join("plots", "ratio_analysis_individual")
        for size in board_sizes:
            plot_avg_time_vs_ratio_individual(results_ns_fixed, size, "Ns", save_folder=plot_folder_ratio_individual) # Ns fixed plots
            plot_avg_time_vs_ratio_individual(results_nl_fixed, size, "Nl", save_folder=plot_folder_ratio_individual) # Nl fixed plots

        # Faceted ratio plots:
        plot_folder_ratio = os.path.join("plots", "ratio_analysis")
        plot_avg_time_vs_ratio_faceted(results_ns_fixed, results_nl_fixed, board_sizes, save_folder=plot_folder_ratio, filename="avg_time_vs_ratio_faceted.png") # Keep avg time faceted plot
        plot_win_prob_vs_ratio_faceted(results_ns_fixed, results_nl_fixed, board_sizes, save_folder=plot_folder_ratio, filename="win_prob_vs_ratio_faceted.png") # Re-introduce faceted win prob plot
        plot_win_prob_vs_ratio_separated_board_sizes(results_ns_fixed, results_nl_fixed, board_sizes, save_folder=plot_folder_ratio, filename_prefix="win_prob_vs_ratio_board_sizes") # Keep multiple lines plot

        # --- Save raw data to CSV in base 'plots' folder ---
        df_ns_fixed.to_csv("plots/results_ns_fixed.csv", index=False) # Now df_ns_fixed is defined here
        df_nl_fixed.to_csv("plots/results_nl_fixed.csv", index=False) # Now df_nl_fixed is defined here
        print("Raw data (Ns fixed) saved to: plots/results_ns_fixed.csv")
        print("Raw data (Nl fixed) saved to: plots/results_nl_fixed.csv")

    else: # Otherwise just show plots
        for size in board_sizes:
              plot_avg_time_vs_ratio_individual(results_ns_fixed, size, "Ns")  # Ns fixed
              plot_avg_time_vs_ratio_individual(results_nl_fixed, size, "Nl")  # Nl fixed
        plot_win_prob_vs_ratio_separated_board_sizes(results_ns_fixed, results_nl_fixed, board_sizes) # Keep multiple lines plot
        plot_avg_time_vs_board_size_separated(results_ns_fixed, results_nl_fixed)
        plot_win_prob_vs_board_size_separated(results_ns_fixed, results_nl_fixed)
        plot_win_prob_vs_ratio_faceted(results_ns_fixed, results_nl_fixed, board_sizes) # Re-introduce faceted win prob plot

    return results_ns_fixed, results_nl_fixed

In [3]:
def plot_board_heatmap_representation_with_arrows(board_size, snakes, ladders, board_linear_size, save_folder=None, filename="board_layout.png"):
    """
    Plots board representation with heatmap, labels, straight arrows, and SAVES DATA TO CSV.
    Colours snakes red, ladders green. Returns Figure object.
    """
    grid_representation = create_labeled_board_representation(board_size, snakes, ladders, board_linear_size)
    grid_size = board_linear_size
    empty_board_data = np.zeros((grid_size, grid_size))

    fig, ax = plt.subplots(figsize=(12, 10))

    # Colour map for snakes and ladders (explicit colours)
    cmap = sns.color_palette(['lightgray', 'red', 'green']) # 0: default, 1: snake, 2: ladder
    board_data_coloured = np.zeros((grid_size, grid_size)) # Initialize colour data
    annot_data = np.empty_like(grid_representation, dtype=object) # Annotation data

    for tile in range(1, board_size + 1): # Iterate to colour snakes and ladders
        row = (tile - 1) // grid_size
        col = (tile - 1) % grid_size
        if row % 2 == 1:
            col = (grid_size - 1) - col
        
        tile_label = str(tile)
        colour_index = 0 # Default colour index (lightgray)

        if tile in snakes:
            tile_label = f'S{tile}→{snakes[tile]}'
            colour_index = 1 # Snake colour index (red)
        elif tile in ladders:
            tile_label = f'L{tile}→{ladders[tile]}'
            colour_index = 2 # Ladder colour index (green)
        
        board_data_coloured[grid_size - 1 - row, col] = colour_index # Assign colour index
        annot_data[grid_size - 1 - row, col] = tile_label # Assign label


    sns.heatmap(board_data_coloured, annot=annot_data, fmt='s', cmap=cmap, # Use board_data_coloured and cmap
                linewidths=1, linecolor='black', cbar=False, annot_kws={"fontsize": 8, "fontweight": "bold"}, ax=ax)


    ax.set_title(f"Snakes and Ladders Board ({board_size} Tiles) - {board_linear_size}x{board_linear_size}", fontsize=14) # Removed "with Arrows" from title
    ax.set_xticks([])
    ax.set_yticks([])
    ax.invert_yaxis()


    # --- Straight Arrows, Red for Snakes, Green for Ladders ---
    arrow_props_snake=dict(arrowstyle='-|>', mutation_scale=15, lw=1.5, color='red', connectionstyle="arc3,rad=0.") # Straight arrows for snakes, thicker lines
    arrow_props_ladder=dict(arrowstyle='-|>', mutation_scale=15, lw=1.5, color='green', connectionstyle="arc3,rad=0.") # Straight arrows for ladders, thicker lines


    for start_tile, end_tile in snakes.items():
        start_pos = _tile_to_position_dynamic_grid(start_tile, board_linear_size)
        end_pos = _tile_to_position_dynamic_grid(end_tile, board_linear_size)
        arrow = FancyArrowPatch(start_pos, end_pos, transform=ax.transData, **arrow_props_snake)
        ax.add_patch(arrow)

    for start_tile, end_tile in ladders.items():
        start_pos = _tile_to_position_dynamic_grid(start_tile, board_linear_size)
        end_pos = _tile_to_position_dynamic_grid(end_tile, board_linear_size)
        arrow = FancyArrowPatch(start_pos, end_pos, transform=ax.transData, **arrow_props_ladder)
        ax.add_patch(arrow)
    # --- End Arrows ---


    if save_folder:
        os.makedirs(save_folder, exist_ok=True)
        filepath_png = os.path.join(save_folder, filename)
        fig.savefig(filepath_png, bbox_inches='tight')
        print(f"Board layout plot saved to: {filepath_png}")

        # --- Save board layout data to CSV ---
        filepath_csv = os.path.join(save_folder, filename.replace(".png", ".csv")) # CSV filename
        with open(filepath_csv, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(['Tile', 'Type', 'Start', 'End']) # Header row
            for tile in range(1, board_size + 1):
                entity_type = "Tile" # Default entity type
                start_point = end_point = ""
                if tile in snakes:
                    entity_type = "Snake"
                    start_point = tile
                    end_point = snakes[tile]
                elif tile in ladders:
                    entity_type = "Ladder"
                    start_point = tile
                    end_point = ladders[tile]
                csv_writer.writerow([tile, entity_type, start_point, end_point])
        print(f"Board layout data saved to: {filepath_csv}") # Confirmation message


    else:
        plt.show()
    plt.close(fig)

    return fig # Return Figure object


def plot_turn_distribution(turn_counts_simulation, turn_counts_markov, num_snakes, num_ladders, num_simulations, save_folder=None, filename="turn_distribution.png"):
    """
    Plots and compares turn distributions from simulations and Markov model, with save functionality.
    """
    fig = plt.figure(figsize=(10, 6)) # Create figure object

    # Determine bin size (same as before)
    max_turns = max(max(turn_counts_simulation), max(turn_counts_markov))
    min_turns = min(min(turn_counts_simulation), min(turn_counts_markov))
    bin_size = max(1, (max_turns - min_turns) // 20)

    # Plot histograms (same as before)
    plt.hist(turn_counts_simulation, bins=range(min_turns, max_turns + bin_size, bin_size),
             align='left', rwidth=0.8, color='skyblue', edgecolor='black', density=True, label='Simulation')
    plt.hist(turn_counts_markov, bins=range(min_turns, max_turns + bin_size, bin_size),
             align='left', rwidth=0.8, color='coral', edgecolor='black', density=True, alpha=0.7, label='Markov Model')

    plt.xlabel("Number of Turns")
    plt.ylabel("Probability")
    plt.title(f"Comparison of Turn Distributions\n(Simulation vs. Markov Model) - ({num_snakes} Snakes, {num_ladders} Ladders, {num_simulations} Simulations)")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.xticks(range(min_turns, max_turns + 1, bin_size*2))
    plt.legend()
    plt.tight_layout()

    if save_folder: # Save plot if save_folder is provided
        os.makedirs(save_folder, exist_ok=True)
        filepath = os.path.join(save_folder, filename)
        plt.savefig(filepath, bbox_inches='tight')
        print(f"Turn distribution plot saved to: {filepath}")
    else:
        plt.show() # Show plot if not saving
    plt.close(fig) # Close figure


def plot_board_heatmap_representation_with_arrows(board_size, snakes, ladders, board_linear_size, save_folder=None, filename="board_layout.png"):
    """
    Plots a Snakes and Ladders board representation using sns.heatmap with labels AND arrows,
    dynamically sized, with save functionality.
    **Corrected to return the Figure object.**
    """
    grid_representation = create_labeled_board_representation(board_size, snakes, ladders, board_linear_size)
    grid_size = board_linear_size
    empty_board_data = np.zeros((grid_size, grid_size))

    fig, ax = plt.subplots(figsize=(12, 10)) # Create Figure and Axes objects

    sns.heatmap(empty_board_data, annot=grid_representation, fmt='s', cmap='Greys',
                linewidths=1, linecolor='black', cbar=False, annot_kws={"fontsize": 8, "fontweight": "bold"}, ax=ax)

    ax.set_title(f"Snakes and Ladders Board ({board_size} Tiles) with Arrows - {board_linear_size}x{board_linear_size}", fontsize=14)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.invert_yaxis()

    # --- Add Arrows (same as before) ---
    arrow_props_snake=dict(arrowstyle='-|>', mutation_scale=15, lw=2, color='red', connectionstyle='arc3,rad=-0.2')
    arrow_props_ladder=dict(arrowstyle='-|>', mutation_scale=15, lw=2, color='green', connectionstyle='arc3,rad=0.2')

    for start_tile, end_tile in snakes.items():
        start_pos = _tile_to_position_dynamic_grid(start_tile, board_linear_size)
        end_pos = _tile_to_position_dynamic_grid(end_tile, board_linear_size)
        arrow = FancyArrowPatch(start_pos, end_pos, transform=ax.transData, **arrow_props_snake)
        ax.add_patch(arrow)

    for start_tile, end_tile in ladders.items():
        start_pos = _tile_to_position_dynamic_grid(start_tile, board_linear_size)
        end_pos = _tile_to_position_dynamic_grid(end_tile, board_linear_size)
        arrow = FancyArrowPatch(start_pos, end_pos, transform=ax.transData, **arrow_props_ladder)
    # --- End Arrows ---

    if save_folder:
        os.makedirs(save_folder, exist_ok=True)
        filepath = os.path.join(save_folder, filename)
        fig.savefig(filepath, bbox_inches='tight') # Save the Figure object
        print(f"Board layout plot saved to: {filepath}")
    else:
        plt.show()
    plt.close(fig) # Close the Figure object

    return fig # Return the Figure object

def plot_entity_trigger_heatmap(all_entity_triggers, board_size, snakes, ladders, num_simulations, save_folder=None, filename="entity_trigger_heatmap.png"):
    """
    Creates a heatmap showing frequency of snake and ladder triggers, with save functionality.
    """
    fig = plt.figure(figsize=(12, 10)) # Create figure
    tile_counts = Counter()
    for triggers in all_entity_triggers:
        tile_counts.update(triggers)

    board_grid = np.zeros((10, 10))

    # Fill the grid with trigger counts (same as before)
    for tile in range(1, board_size + 1):
        row = (tile - 1) // 10
        col = (tile - 1) % 10
        if row % 2 == 1:
            col = 9 - col
        board_grid[9 - row, col] = tile_counts.get(tile, 0)

    sns.heatmap(board_grid, annot=True, fmt=".0f", cmap="YlOrRd",
                linewidths=.5, cbar_kws={'label': 'Trigger Frequency'},
                mask=(board_grid == 0))
    plt.gca().patch.set_facecolor('lightgray')

    # Add markers for snakes and ladders (same as before)
    for start, end in snakes.items():
        row = (start - 1) // 10
        col = (start - 1) % 10 if row % 2 == 0 else 9 - ((start - 1) % 10)
        plt.text(col, 9 - row, f'S{start}→{end}', color='darkblue',
                ha='center', va='center', fontsize=8, fontweight='bold')

    for start, end in ladders.items():
        row = (start - 1) // 10
        col = (start - 1) % 10 if row % 2 == 0 else 9 - ((start - 1) % 10)
        plt.text(col, 9 - row, f'L{start}→{end}', color='darkgreen',
                ha='center', va='center', fontsize=8, fontweight='bold')

    plt.title(f"Snake and Ladder Trigger Frequency\n({num_simulations} Simulations)")
    plt.xticks([])
    plt.yticks([])
    plt.gca().invert_yaxis()
    plt.tight_layout()

    if save_folder: # Save plot if save_folder is provided
        os.makedirs(save_folder, exist_ok=True)
        filepath = os.path.join(save_folder, filename)
        plt.savefig(filepath, bbox_inches='tight')
        print(f"Entity trigger heatmap saved to: {filepath}")
    else:
        plt.show() # Show plot if not saving
    plt.close(fig) # Close figure


def plot_steady_state_heatmap(steady_state_distribution, board_size, snakes, ladders, save_folder=None, filename="steady_state_heatmap.png"):
    """
    Generates and displays a heatmap of steady-state tile occupation probabilities, with save functionality.
    """
    fig = plt.figure(figsize=(12, 10)) # Create figure
    board_grid_steady_state = np.zeros((10, 10))

    # Get valid states (same as before)
    excluded_tiles = set(snakes.keys()) | set(ladders.keys())
    valid_states = [state for state in range(board_size + 1) if state not in excluded_tiles]

    for tile in valid_states:
        if tile > 0 and tile <= board_size:
            row = (tile - 1) // 10
            col = (tile - 1) % 10
            if row % 2 == 1:
                col = 9 - col
            board_grid_steady_state[9 - row, col] = steady_state_distribution[tile]

    sns.heatmap(board_grid_steady_state, cmap="viridis", linewidths=.5, cbar_kws={'label': 'Steady-State Probability'})
    plt.title(f"Steady-State Tile Occupation Probability Heatmap")
    plt.xticks([])
    plt.yticks([])
    plt.gca().invert_yaxis()
    plt.tight_layout()

    if save_folder: # Save plot if save_folder provided
        os.makedirs(save_folder, exist_ok=True)
        filepath = os.path.join(save_folder, filename)
        plt.savefig(filepath, bbox_inches='tight')
        print(f"Steady-state heatmap saved to: {filepath}")
    else:
        plt.show() # Show plot if not saving
    plt.close(fig) # Close figure


def plot_avg_time_vs_board_size_separated(results_ns_fixed, results_nl_fixed, save_folder="plots/board_scaling", filename_prefix="avg_time_vs_board_size"):
    """
    Plots Average Game Time vs Board Size, separated for Ns-fixed and Nl-fixed scenarios, with save functionality.
    """
    df_ns_fixed = pd.DataFrame(results_ns_fixed)
    df_nl_fixed = pd.DataFrame(results_nl_fixed)

    # --- Plot for Ns Fixed ---
    fig = plt.figure(figsize=(10, 5))
    plt.plot(df_ns_fixed['board_size'], df_ns_fixed['avg_time_sim'], marker='o', linestyle='-', label='Simulation', color='blue')
    plt.plot(df_ns_fixed['board_size'], df_ns_fixed['avg_time_markov'], marker='x', linestyle='--', label='Markov Model', color='blue')

    plt.xlabel("Board Size (Linear Dimension)")
    plt.ylabel("Average Game Time (Turns)")
    plt.title("Average Game Time vs Board Size (Ns Fixed)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    if save_folder:
        ns_fixed_folder = os.path.join(save_folder, "ns_fixed") # Create subfolder for Ns fixed
        os.makedirs(ns_fixed_folder, exist_ok=True)
        filepath = os.path.join(ns_fixed_folder, filename_prefix + "_ns_fixed.png") # Add _ns_fixed to filename
        plt.savefig(filepath, bbox_inches='tight')
        print(f"Average game time vs board size (Ns fixed) plot saved to: {filepath}")
    else:
        plt.show()
    plt.close(fig)


    # --- Plot for Nl Fixed ---
    fig = plt.figure(figsize=(10, 5))
    plt.plot(df_nl_fixed['board_size'], df_nl_fixed['avg_time_sim'], marker='o', linestyle='-', label='Simulation', color='green')
    plt.plot(df_nl_fixed['board_size'], df_nl_fixed['avg_time_markov'], marker='x', linestyle='--', label='Markov Model', color='green')

    plt.xlabel("Board Size (Linear Dimension)")
    plt.ylabel("Average Game Time (Turns)")
    plt.title("Average Game Time vs Board Size (Nl Fixed)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    if save_folder:
        nl_fixed_folder = os.path.join(save_folder, "nl_fixed") # Create subfolder for Nl fixed
        os.makedirs(nl_fixed_folder, exist_ok=True)
        filepath = os.path.join(nl_fixed_folder, filename_prefix + "_nl_fixed.png") # Add _nl_fixed to filename
        plt.savefig(filepath, bbox_inches='tight')
        print(f"Average game time vs board size (Nl fixed) plot saved to: {filepath}")
    else:
        plt.show()
    plt.close(fig)


def plot_win_prob_vs_board_size_separated(results_ns_fixed, results_nl_fixed, save_folder="plots/board_scaling", filename_prefix="win_prob_vs_board_size"):
    """
    Plots Win Probability vs Board Size, separated for Ns-fixed and Nl-fixed scenarios (N/2, N/3, N/4 turns), with save functionality.
    """
    df_ns_fixed = pd.DataFrame(results_ns_fixed)
    df_nl_fixed = pd.DataFrame(results_nl_fixed)

    # --- Plot for Ns Fixed ---
    fig = plt.figure(figsize=(10, 5))
    plt.plot(df_ns_fixed['board_size'], df_ns_fixed['prob_win_n2_sim'], marker='o', linestyle='-', label='Win Prob (N/2 Turns)', color='blue')
    plt.plot(df_ns_fixed['board_size'], df_ns_fixed['prob_win_n3_sim'], marker='^', linestyle='--', label='Win Prob (N/3 Turns)', color='blue')
    plt.plot(df_ns_fixed['board_size'], df_ns_fixed['prob_win_n4_sim'], marker='s', linestyle=':', label='Win Prob (N/4 Turns)', color='blue')

    plt.xlabel("Board Size (Linear Dimension)")
    plt.ylabel("Probability of Winning within N Turns")
    plt.title("Win Probability vs Board Size (Ns Fixed)")
    plt.legend(loc='upper right', fontsize='small')
    plt.grid(True)
    plt.tight_layout()

    if save_folder:
        ns_fixed_folder = os.path.join(save_folder, "ns_fixed") # Subfolder for Ns fixed
        os.makedirs(ns_fixed_folder, exist_ok=True)
        filepath = os.path.join(ns_fixed_folder, filename_prefix + "_ns_fixed.png") # Add _ns_fixed to filename
        plt.savefig(filepath, bbox_inches='tight')
        print(f"Win probability vs board size (Ns fixed) plot saved to: {filepath}")
    else:
        plt.show()
    plt.close(fig)


    # --- Plot for Nl Fixed ---
    fig = plt.figure(figsize=(10, 5))
    plt.plot(df_nl_fixed['board_size'], df_nl_fixed['prob_win_n2_sim'], marker='o', linestyle='-', label='Win Prob (N/2 Turns)', color='green')
    plt.plot(df_nl_fixed['board_size'], df_nl_fixed['prob_win_n3_sim'], marker='^', linestyle='--', label='Win Prob (N/3 Turns)', color='green')
    plt.plot(df_nl_fixed['board_size'], df_nl_fixed['prob_win_n4_sim'], marker='s', linestyle=':', label='Win Prob (N/4 Turns)', color='green')

    plt.xlabel("Board Size (Linear Dimension)")
    plt.ylabel("Probability of Winning within N Turns")
    plt.title("Win Probability vs Board Size (Nl Fixed)")
    plt.legend(loc='upper right', fontsize='small')
    plt.grid(True)
    plt.tight_layout()

    if save_folder:
        nl_fixed_folder = os.path.join(save_folder, "nl_fixed") # Subfolder for Nl fixed
        os.makedirs(nl_fixed_folder, exist_ok=True)
        filepath = os.path.join(nl_fixed_folder, filename_prefix + "_nl_fixed.png") # Add _nl_fixed to filename
        plt.savefig(filepath, bbox_inches='tight')
        print(f"Win probability vs board size (Nl fixed) plot saved to: {filepath}")
    else:
        plt.show()
    plt.close(fig)



def plot_avg_time_vs_ratio_faceted(results_ns_fixed, results_nl_fixed, board_sizes, save_folder="plots/ratio_analysis", filename="avg_time_vs_ratio_faceted.png"):
    """
    Plots Faceted Avg Game Time vs Ns/Nl Ratio, SAVES DATA TO CSV.
    Increased line width, distinct markers.
    """
    df_ns_fixed = pd.DataFrame(results_ns_fixed)
    df_nl_fixed = pd.DataFrame(results_nl_fixed)

    fig, axes = plt.subplots(2, len(board_sizes), figsize=(15, 8), sharey=True)

    # Line styles and markers - defined here for consistency
    line_width = 2 # Thicker lines
    markers = ['o', 'x'] # Circle for Simulation, Cross for Markov

    for i, size in enumerate(sorted(board_sizes)):
        # --- Ns Fixed Facet ---
        ax_ns = axes[0, i]
        df_subset_ns = df_ns_fixed[df_ns_fixed['board_size'] == size]
        ax_ns.plot(df_subset_ns['ratio'], df_subset_ns['avg_time_sim'], marker=markers[0], linestyle='-', label='Simulation', color='blue', linewidth=line_width, markersize=8) # Increased linewidth, markersize
        ax_ns.plot(df_subset_ns['ratio'], df_subset_ns['avg_time_markov'], marker=markers[1], linestyle='--', label='Markov', color='blue', linewidth=line_width, markersize=8) # Increased linewidth, markersize
        ax_ns.set_title(f'Board Size: {size}x{size} (Ns Fixed)')
        ax_ns.set_xlabel("Ns/Nl Ratio")
        ax_ns.grid(True)
        ax_ns.set_ylim(bottom=0) # Ensure y-axis starts at 0
        if i == 0:
            ax_ns.set_ylabel("Avg Game Time")
            ax_ns.legend(fontsize='small')

        # --- Nl Fixed Facet ---
        ax_nl = axes[1, i]
        df_subset_nl = df_nl_fixed[df_nl_fixed['board_size'] == size]
        ax_nl.plot(df_subset_nl['ratio'], df_subset_nl['avg_time_sim'], marker=markers[0], linestyle='-', label='Simulation', color='green', linewidth=line_width, markersize=8) # Increased linewidth, markersize
        ax_nl.plot(df_subset_nl['ratio'], df_subset_nl['avg_time_markov'], marker=markers[1], linestyle='--', label='Markov', color='green', linewidth=line_width, markersize=8) # Increased linewidth, markersize

        ax_nl.set_title(f'Board Size: {size}x{size} (Nl Fixed)')
        ax_nl.set_xlabel("Ns/Nl Ratio")
        ax_nl.grid(True)
        ax_nl.set_ylim(bottom=0) # Ensure y-axis starts at 0
        if i == 0:
            ax_nl.set_ylabel("Avg Game Time")
            ax_nl.legend(fontsize='small')


    fig.suptitle("Average Game Time vs Ns/Nl Ratio, Faceted by Board Size", fontsize=14)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

    if save_folder:
        os.makedirs(save_folder, exist_ok=True)
        filepath_png = os.path.join(save_folder, filename)
        fig.savefig(filepath_png, bbox_inches='tight')
        print(f"Faceted average game time vs ratio plot saved to: {filepath_png}")

        # --- Save raw data to CSV ---
        filepath_csv = os.path.join(save_folder, filename.replace(".png", ".csv"))
        with open(filepath_csv, 'w', newline='') as csvfile:
            csv_writer = csv.writer(csvfile)
            # Write header row
            csv_writer.writerow(['Board Size', 'Ratio', 'Entity Fixed', 'Data Source', 'Average Game Time'])

            # Write data rows for Ns Fixed
            for index, row in df_ns_fixed.iterrows():
                csv_writer.writerow([row['board_size'], row['ratio'], 'Ns Fixed', 'Simulation', row['avg_time_sim']])
                csv_writer.writerow([row['board_size'], row['ratio'], 'Ns Fixed', 'Markov Model', row['avg_time_markov']])

            # Write data rows for Nl Fixed
            for index, row in df_nl_fixed.iterrows():
                csv_writer.writerow([row['board_size'], row['ratio'], 'Nl Fixed', 'Simulation', row['avg_time_sim']])
                csv_writer.writerow([row['board_size'], row['ratio'], 'Nl Fixed', 'Markov Model', row['avg_time_markov']])
        print(f"Raw data for faceted average game time vs ratio plot saved to: {filepath_csv}") # Confirmation message


    else:
        plt.show()
    plt.close(fig)

def plot_win_prob_vs_ratio_faceted(results_ns_fixed, results_nl_fixed, board_sizes, save_folder="plots/ratio_analysis", filename="win_prob_vs_ratio_faceted.png"):
    """
    Plots Win Probability vs Ns/Nl Ratio, faceted by board size.
    Separate rows for Ns-fixed and Nl-fixed, and separate lines for N/2, N/3, N/4.
    Includes line width, marker, and data saving improvements.
    """
    df_ns_fixed = pd.DataFrame(results_ns_fixed)
    df_nl_fixed = pd.DataFrame(results_nl_fixed)

    win_fractions = [0.5, 1/3, 0.25]
    fraction_labels = {'0.5': 'N/2', '0.3333333333333333': 'N/3', '0.25': 'N/4'}
    line_styles = ['-', '--', ':'] # Line styles for win conditions
    markers = ['o', '^', 's'] # Markers for win conditions

    fig, axes = plt.subplots(2, len(board_sizes), figsize=(18, 8), sharey=True) # Adjusted figsize

    for i, size in enumerate(sorted(board_sizes)):
        # --- Ns Fixed Facet ---
        ax_ns = axes[0, i]
        df_subset_ns = df_ns_fixed[df_ns_fixed['board_size'] == size]
        for j, fraction in enumerate(win_fractions):  # Use enumerate for line style/marker indexing
            prob_col_name = f'prob_win_{fraction_labels[str(fraction)]}_sim'.lower().replace('/', '').replace('.', '')
            ax_ns.plot(df_subset_ns['ratio'], df_subset_ns[prob_col_name], marker=markers[j], linestyle=line_styles[j], label=f'N/{fraction_labels[str(fraction)]} Turns (Sim)', linewidth=2, markersize=8, color='blue') # Consistent colours, thicker lines

        ax_ns.set_title(f'Board Size: {size}x{size} (Ns Fixed)')
        ax_ns.set_xlabel("Ns/Nl Ratio")
        ax_ns.grid(True)
        ax_ns.set_ylim(bottom=0, top=1.05) #Consistent y-axis
        if i == 0:
            ax_ns.set_ylabel("Win Probability")
            ax_ns.legend(fontsize='small')

        # --- Nl Fixed Facet ---
        ax_nl = axes[1, i]
        df_subset_nl = df_nl_fixed[df_nl_fixed['board_size'] == size]
        for j, fraction in enumerate(win_fractions):  # Use enumerate for line style/marker indexing
            prob_col_name = f'prob_win_{fraction_labels[str(fraction)]}_sim'.lower().replace('/', '').replace('.', '')
            ax_nl.plot(df_subset_nl['ratio'], df_subset_nl[prob_col_name], marker=markers[j], linestyle=line_styles[j], label=f'N/{fraction_labels[str(fraction)]} Turns (Sim)', linewidth=2, markersize=8, color='green') # Consistent colours, thicker lines

        ax_nl.set_title(f'Board Size: {size}x{size} (Nl Fixed)')
        ax_nl.set_xlabel("Ns/Nl Ratio")
        ax_nl.grid(True)
        ax_nl.set_ylim(bottom=0, top=1.05) # Consistent Y-axis
        if i == 0:
            ax_nl.set_ylabel("Win Probability")
            ax_nl.legend(fontsize='small')


    fig.suptitle("Win Probability vs Ns/Nl Ratio, Faceted by Board Size", fontsize=14)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])

    if save_folder:
        os.makedirs(save_folder, exist_ok=True)
        filepath = os.path.join(save_folder, filename)
        plt.savefig(filepath, bbox_inches='tight')
        print(f"Faceted win probability vs ratio plot saved to: {filepath}")

        # --- Save raw data to CSV (Optional - for faceted plots) ---
        # You *could* save a combined CSV here, but it might be less useful
        # than the separate CSVs saved by run_board_scaling_analysis.
        # Consider if you *really* need this combined CSV.

    else:
        plt.show()
    plt.close(fig)

In [ ]:
# %%
def create_snakes_and_ladders(board_size, num_snakes, num_ladders, max_regeneration_attempts=500): # Added max_regeneration_attempts
    """
    Generates random positions, with regeneration attempt limit.
    """
    attempts = 0 # Initialize attempt counter
    while attempts < max_regeneration_attempts:  # Loop with attempt limit
        attempts += 1
        snakes, ladders = _generate_snakes_and_ladders_once(board_size, num_snakes, num_ladders)

        # Connectivity check (same as before)
        transition_matrix = create_transition_matrix(board_size, snakes, ladders)
        excluded_tiles = set(snakes.keys()) | set(ladders.keys())
        valid_states = [state for state in range(board_size + 1) if state not in excluded_tiles]
        n = len(valid_states)
        Q_check = transition_matrix.iloc[:n, :n].copy()

        has_zero_row = False
        for index, row in Q_check.iterrows():
            if all(row == 0):
                has_zero_row = True
                break

        if not has_zero_row:
            print(f"Valid board generated after {attempts} regeneration attempts.") # Show attempts
            return snakes, ladders
        else:
            print(f"Board layout with isolated states. Regenerating... (Attempt {attempts}/{max_regeneration_attempts})") # Show attempt count

    raise Exception(f"Failed to generate valid board after {max_regeneration_attempts} attempts.") # Raise exception if limit reached
    # OR, alternatively, return None: return None, None # Signal failure to generate board

def _generate_snakes_and_ladders_once(board_size, num_snakes, num_ladders):
    """
    Helper function to generate snakes and ladders with connectivity heuristics.
    """
    snakes = {}
    ladders = {}
    used_tiles = set()

    def generate_endpoints(entity_type, used_tiles):
        attempts = 0
        while attempts < 100:  # Limit attempts
            attempts += 1
            if entity_type == "snake":
                start_tile = random.randint(20, board_size - 1)  # Snakes start further up
            else:  # ladder
                start_tile = random.randint(2, board_size - 30) # Ladders start lower

            if start_tile in used_tiles:
                continue

            if entity_type == "snake":
                # Corrected line: Ensure end_tile range is valid
                end_tile = random.randint(1, max(1, start_tile - 20)) # Snakes go down significantly, ensure valid range
            else:  # ladder
                end_tile = random.randint(start_tile + 10, board_size - 1) # Ladders go up significantly

            if end_tile in used_tiles or end_tile == start_tile:
                continue

            valid_placement = _check_overlap(snakes, ladders, start_tile, end_tile, entity_type) # Use helper

            if valid_placement:
                used_tiles.add(start_tile)
                used_tiles.add(end_tile)
                return start_tile, end_tile
        return None, None # Return None if no valid placement found after attempts


    def _check_overlap(snakes, ladders, start_tile, end_tile, entity_type):
        """Helper function to check for overlaps (same as before)."""
        valid_placement = True
        for s_start, s_end in snakes.items():
            if start_tile == s_start or start_tile == s_end or end_tile == s_start or end_tile == s_end:
                valid_placement = False
                break
        if not valid_placement:
            return False

        for l_start, l_end in ladders.items():
            if start_tile == l_start or start_tile == l_end or end_tile == l_start or end_tile == l_end:
                valid_placement = False
                break
        return valid_placement


    # Generate Snakes - with retry mechanism if placement fails
    snakes_placed = 0
    for _ in range(num_snakes * 2): # Try placing snakes more times than needed
        if snakes_placed >= num_snakes:
            break # Stop if enough snakes placed
        start, end = generate_endpoints("snake", used_tiles)
        if start is not None: # Check if valid placement was found
            snakes[start] = end
            snakes_placed += 1

    # Generate Ladders - with retry mechanism
    ladders_placed = 0
    for _ in range(num_ladders * 2): # Try placing ladders more times than needed
        if ladders_placed >= num_ladders:
            break # Stop if enough ladders placed
        start, end = generate_endpoints("ladder", used_tiles)
        if start is not None: # Check if valid placement was found
            ladders[start] = end
            ladders_placed += 1

    print("\n--- Board Generation (Debugging) ---") # Debugging print
    print("Number of snakes placed:", len(snakes))
    print("Number of ladders placed:", len(ladders))
    print("Snakes:", snakes)
    print("Ladders:", ladders)
    print("--- End Board Generation (Debugging) ---")
    return snakes, ladders



# %%
def simulate_game(board_size, snakes, ladders, record_positions=False, record_entity_triggers=False):
    """
    Simulates a single game of Snakes and Ladders.

    Args:
        board_size (int): Size of the board
        snakes (dict): Dictionary of snake positions
        ladders (dict): Dictionary of ladder positions
        record_positions (bool): Whether to record all positions visited
        record_entity_triggers (bool): Whether to record snake/ladder triggers

    Returns:
        tuple: (turns taken, positions visited, entity triggers)
    """
    position = 0
    turns = 0
    positions_visited = [] if record_positions else None
    entity_triggers = [] if record_entity_triggers else None

    if record_positions:
        positions_visited.append(position)

    while position < board_size:
        roll = random.randint(1, 6)
        new_position = position + roll

        if new_position > board_size:
            new_position = board_size - (new_position - board_size)

        position = new_position
        turns += 1

        if record_positions:
            positions_visited.append(position)

        if position in snakes:
            if record_entity_triggers:
                entity_triggers.append(position)
            position = snakes[position]
            if record_positions:
                positions_visited.append(position)
        elif position in ladders:
            if record_entity_triggers:
                entity_triggers.append(position)
            position = ladders[position]
            if record_positions:
                positions_visited.append(position)

    return turns, positions_visited, entity_triggers

# %%
def create_transition_matrix(board_size, snakes, ladders):
    """
    Creates a transition matrix for the Snakes and Ladders Markov model,
    EXCLUDING snake heads and ladder bases as states.

    Args:
        board_size (int): Size of the board
        snakes (dict): Dictionary of snake positions (head: tail)
        ladders (dict): Dictionary of ladder positions (base: top)

    Returns:
        pandas.DataFrame: Transition matrix (DataFrame)
    """
    # Identify snake heads and ladder bases to exclude as states
    excluded_tiles = set(snakes.keys()) | set(ladders.keys())
    valid_states = [state for state in range(board_size + 1) if state not in excluded_tiles]
    num_states = len(valid_states)

    transition_matrix = pd.DataFrame(0.0, index=valid_states, columns=valid_states)

    state_to_index = {state: index for index, state in enumerate(valid_states)} # Map state to index

    for current_tile in valid_states:
        if current_tile == board_size: # Absorbing state
            transition_matrix.loc[current_tile, current_tile] = 1.0
            continue

        probabilities = [0.0] * num_states # Initialize probabilities for valid states

        for roll in range(1, 7):
            next_position_tentative = current_tile + roll

            if next_position_tentative > board_size:
                next_position_tentative = board_size - (next_position_tentative - board_size) # Bounce back

            next_position = next_position_tentative # Initialize assuming no snake/ladder

            # Check for snakes and ladders at the *tentative* next position
            if next_position_tentative in snakes:
                next_position = snakes[next_position_tentative] # Move to snake tail
            elif next_position_tentative in ladders:
                next_position = ladders[next_position_tentative] # Move to ladder top

            # Ensure next_position is a valid state (not a snake head or ladder base)
            while next_position in excluded_tiles:
                if next_position in snakes:
                    next_position = snakes[next_position] # Chain snake
                elif next_position in ladders:
                    next_position = ladders[next_position] # Chain ladder
                else: # Should not happen in a well-formed board, but as a safeguard
                    break # Break out if stuck in an infinite loop

            if next_position in valid_states:
                next_state_index = state_to_index[next_position] # Get index in reduced state space
                probabilities[next_state_index] += 1/6.0
            elif next_position == board_size: # Handle reaching the goal (absorbing state)
                next_state_index = state_to_index[board_size]
                probabilities[next_state_index] += 1/6.0


        # Assign probabilities to the transition matrix row
        for next_state_index, prob in enumerate(probabilities):
            next_state = valid_states[next_state_index]
            transition_matrix.loc[current_tile, next_state] = prob


    return transition_matrix

def calculate_expected_turns_markov(transition_matrix, board_size, snakes, ladders):
    # Get valid states (excluding snake heads/ladder bases AND the absorbing state)
    excluded_tiles = set(snakes.keys()) | set(ladders.keys()) | {board_size}
    valid_states = [state for state in range(board_size) if state not in excluded_tiles]
    n = len(valid_states)

    # Extract Q matrix (only transient states)
    Q = transition_matrix.loc[valid_states, valid_states].copy()
    I = pd.DataFrame(np.identity(n), index=valid_states, columns=valid_states)

    try:
        N = np.linalg.inv(I - Q)
        ones = np.ones(n)
        expected_turns = N.dot(ones)
        return expected_turns[0]  # Return the expected turns from state 0
    except np.linalg.LinAlgError as e:
        print("Matrix inversion failed. Check transition matrix structure.")
        raise e

def calculate_steady_state_distribution(transition_matrix, board_size, snakes, ladders, initial_state=0, num_iterations=500): # Added snakes, ladders
    """
    Calculates the steady-state distribution of tile occupation probabilities,
    ADJUSTED FOR REDUCED STATE SPACE.
    """
    # Get valid states - IMPORTANT: Re-calculate valid states to be consistent with transition_matrix
    excluded_tiles = set(snakes.keys()) | set(ladders.keys())
    valid_states = [state for state in range(board_size + 1) if state not in excluded_tiles]
    num_states = len(valid_states) # Recalculate num_states based on valid_states

    state_distribution = pd.Series([0.0] * num_states, index=valid_states) # Use valid_states as index
    state_distribution[initial_state] = 1.0

    # Ensure state_distribution index matches transition_matrix columns - EXPLICIT CHECK
    state_distribution.index = transition_matrix.columns # Force index alignment (although now it should match already)

    # Debugging Prints:
    print("\n--- Debugging Information (calculate_steady_state_distribution) ---")
    print("Shape of state_distribution (should be num_states):", state_distribution.shape)
    print("Shape of transition_matrix (should be num_states x num_states):", transition_matrix.shape)
    print("\nstate_distribution Index (first 10):", state_distribution.index[:10]) # Print first 10 indices
    print("\ntransition_matrix Columns Index (first 10):", transition_matrix.columns[:10]) # Print first 10 column indices
    print("\nstate_distribution dtype:", state_distribution.dtype)
    print("\ntransition_matrix dtypes (first 5 columns):", transition_matrix.dtypes.head())
    print("--- End Debugging Information ---")


    for _ in range(num_iterations):
        # state_distribution = state_distribution.dot(transition_matrix) # Original dot product
        state_distribution = state_distribution @ transition_matrix # Try @ operator - ALTERNATIVE DOT PRODUCT

    return state_distribution

def simulate_game_markov_distribution(transition_matrix, num_games, board_size):
    """
    Simulates games directly from the Markov transition matrix to generate turn distribution.

    Args:
        transition_matrix (pd.DataFrame): Markov transition matrix.
        num_games (int): Number of games to simulate.
        board_size (int): Size of the board.

    Returns:
        list: List of turn counts for simulated games.
    """

    turn_counts_markov = []
    start_state_index = 0 # Assuming starting from state 0

    for _ in range(num_games):
        current_state = start_state_index
        turns = 0
        while current_state < board_size: # While not in absorbing state (state 100)
            probabilities = transition_matrix.loc[current_state].values # Get transition probabilities for current state
            next_state = random.choices(transition_matrix.columns, weights=probabilities, k=1)[0] # Probabilistic transition
            current_state = next_state
            turns += 1
        turn_counts_markov.append(turns)
    return turn_counts_markov

# --- ADDED FUNCTION ---
def calculate_markov_win_probabilities(transition_matrix, board_size):
    """
    Calculates Markov model win probabilities for N/2, N/3, and N/4 turns.

    Args:
        transition_matrix (pd.DataFrame): Markov transition matrix.
        board_size (int): Size of the board.

    Returns:
        dict: Dictionary of win probabilities for N/2, N/3, N/4 turns.
    """
    win_probabilities_markov = {0.5: 0, 1/3: 0, 0.25: 0}
    initial_state_distribution = pd.Series(0.0, index=transition_matrix.columns)
    initial_state_distribution[0] = 1.0  # Start at state 0

    state_distribution = initial_state_distribution.copy()

    for turns in range(1, board_size + 1):  # Iterate up to board_size turns (should be sufficient)
        state_distribution = state_distribution @ transition_matrix  # Update state distribution

        for fraction in win_probabilities_markov:
            if turns == int(board_size * fraction): # Check for each turn limit
                win_probabilities_markov[fraction] = state_distribution[board_size] # Prob. of being in absorbing state

    return win_probabilities_markov

# %%
def create_labeled_board_representation(board_size, snakes, ladders, board_linear_size):
    """
    Creates a NumPy array representation of a Snakes and Ladders board with labels,
    dynamically sized based on board_linear_size.

    Args:
        board_size (int): Total number of tiles on the board
        snakes (dict): Dictionary of snake positions
        ladders (dict): Dictionary of ladder positions
        board_linear_size (int): Linear dimension of the square board (e.g., 10 for 10x10)

    Returns:
        numpy.ndarray: NumPy array of strings representing tile labels, sized board_linear_size x board_linear_size
    """
    grid_size = board_linear_size
    grid = np.empty((grid_size, grid_size), dtype=object)  # NumPy array of objects, dynamic size

    for tile in range(1, board_size + 1): # Iterate only up to board_size
        row = (tile - 1) // grid_size # Use grid_size here
        col = (tile - 1) % grid_size # Use grid_size here
        if row % 2 == 1:
            col = (grid_size - 1) - col # Use grid_size here

        tile_label = str(tile)

        if tile in snakes:
            tile_label = f'S{tile}→{snakes[tile]}'
        elif tile in ladders:
            tile_label = f'L{tile}→{ladders[tile]}'

        grid[grid_size - 1 - row, col] = tile_label # Use grid_size for row index as well (and invert row)


    return grid


def plot_board_heatmap_representation(board_size, snakes, ladders):
    """
    Plots a Snakes and Ladders board representation using sns.heatmap with labels.

    Args:
        board_size (int): Size of the board
        snakes (dict): Dictionary of snake positions
        ladders (dict): Dictionary of ladder positions
    """
    grid_representation = create_labeled_board_representation(board_size, snakes, ladders)
    empty_board_data = np.zeros((10, 10)) # Dummy data for heatmap

    fig, ax = plt.subplots(figsize=(12, 10))
    sns.heatmap(empty_board_data, annot=grid_representation, fmt='s', cmap='Greys', # Use 'Wistia' or 'Greys' cmap
                linewidths=1, linecolor='black', cbar=False, annot_kws={"fontsize": 9, "fontweight": "bold"}, ax=ax) # Adjust fontsize here

    ax.set_title("Snakes and Ladders Board", fontsize=14)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.invert_yaxis() # Invert y-axis to match board layout

    return plt # Return plt object

def plot_turn_distribution(turn_counts_simulation, turn_counts_markov, num_snakes, num_ladders, num_simulations):
    """
    Plots and compares turn distributions from simulations and Markov model.
    """
    plt.figure(figsize=(10, 6))

    # Determine bin size
    max_turns = max(max(turn_counts_simulation), max(turn_counts_markov))
    min_turns = min(min(turn_counts_simulation), min(turn_counts_markov))

    bin_size = max(1, (max_turns - min_turns) // 20)

    # Plot histograms for both simulation and Markov data
    plt.hist(turn_counts_simulation, bins=range(min_turns, max_turns + bin_size, bin_size),
             align='left', rwidth=0.8, color='skyblue', edgecolor='black', density=True, label='Simulation')
    plt.hist(turn_counts_markov, bins=range(min_turns, max_turns + bin_size, bin_size),
             align='left', rwidth=0.8, color='coral', edgecolor='black', density=True, alpha=0.7, label='Markov Model') # Use different color, reduce alpha

    plt.xlabel("Number of Turns")
    plt.ylabel("Probability")
    plt.title(f"Comparison of Turn Distributions\n(Simulation vs. Markov Model) - ({num_snakes} Snakes, {num_ladders} Ladders, {num_simulations} Simulations)")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.xticks(range(min_turns, max_turns + 1, bin_size*2)) # Adjusted x-ticks for better readability
    plt.legend() # Show legend to distinguish datasets
    plt.tight_layout()
    plt.show()



def plot_board_heatmap(all_positions_visited, board_size, snakes, ladders, num_simulations):
    """
    Creates a heatmap showing frequency of visits to each board position.
    """

    tile_counts = Counter()
    for positions_list in all_positions_visited:
        tile_counts.update(positions_list)

    board_grid = np.zeros((10, 10))
    for tile in range(1, board_size + 1):
        row = (tile - 1) // 10
        col = (tile - 1) % 10
        if row % 2 == 1:
            col = 9 - col
        board_grid[9 - row, col] = tile_counts.get(tile, 0)

    plt.figure(figsize=(12, 10))
    sns.heatmap(board_grid, annot=False, fmt="d", cmap="viridis",
                linewidths=.5, cbar_kws={'label': 'Visit Frequency'})

    # Add snake and ladder markers
    for start, end in snakes.items():
        row = (start - 1) // 10
        col = (start - 1) % 10 if row % 2 == 0 else 9 - ((start - 1) % 10)
        plt.text(col, 9 - row, 'S', color='white', ha='center', va='center', fontsize=8)

    for start, end in ladders.items():
        row = (start - 1) // 10
        col = (start - 1) % 10 if row % 2 == 0 else 9 - ((start - 1) % 10)
        plt.text(col, 9 - row, 'L', color='white', ha='center', va='center', fontsize=8)

    plt.title(f"Visit Frequency Heatmap\n({num_simulations} Simulations)")
    plt.xticks(np.arange(0, 10, 1), labels=[f'{i*10+1}-{(i+1)*10}' for i in range(10)])
    plt.yticks([])
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

def plot_entity_trigger_heatmap(all_entity_triggers, board_size, snakes, ladders, num_simulations):
    """
    Creates a heatmap showing frequency of snake and ladder triggers.
    Fixed to show correct start→end labels for snakes and ladders.
    """
    tile_counts = Counter()
    for triggers in all_entity_triggers:
        tile_counts.update(triggers)

    board_grid = np.zeros((10, 10))

    # Fill the grid with trigger counts
    for tile in range(1, board_size + 1):
        row = (tile - 1) // 10
        col = (tile - 1) % 10
        if row % 2 == 1:
            col = 9 - col
        board_grid[9 - row, col] = tile_counts.get(tile, 0)

    plt.figure(figsize=(12, 10))

    # Create heatmap with white for zero values
    sns.heatmap(board_grid, annot=True, fmt=".0f", cmap="YlOrRd",
                linewidths=.5, cbar_kws={'label': 'Trigger Frequency'},
                mask=(board_grid == 0))

    # Add a background color for cells with zero values
    plt.gca().patch.set_facecolor('lightgray')

    # Add markers for snakes and ladders with correct start→end format
    for start, end in snakes.items():
        row = (start - 1) // 10
        col = (start - 1) % 10 if row % 2 == 0 else 9 - ((start - 1) % 10)
        plt.text(col, 9 - row, f'S{start}→{end}', color='darkblue',
                ha='center', va='center', fontsize=8, fontweight='bold')

    for start, end in ladders.items():
        row = (start - 1) // 10
        col = (start - 1) % 10 if row % 2 == 0 else 9 - ((start - 1) % 10)
        plt.text(col, 9 - row, f'L{start}→{end}', color='darkgreen',
                ha='center', va='center', fontsize=8, fontweight='bold')

    plt.title(f"Snake and Ladder Trigger Frequency\n({num_simulations} Simulations)")
    plt.xticks([])
    plt.yticks([])
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

def plot_steady_state_heatmap(steady_state_distribution, board_size, snakes, ladders): # Added snakes, ladders
    """
    Generates and displays a heatmap of steady-state tile occupation probabilities,
    ADJUSTED FOR REDUCED STATE SPACE.
    """
    plt.figure(figsize=(12, 10))
    board_grid_steady_state = np.zeros((10, 10))

    # Get valid states - IMPORTANT: Use valid states from transition_matrix creation
    excluded_tiles = set(snakes.keys()) | set(ladders.keys())
    valid_states = [state for state in range(board_size + 1) if state not in excluded_tiles]

    for tile in valid_states: # Iterate over valid_states only - CORRECTED LOOP
        if tile > 0 and tile <= board_size: # Ensure tile is within board range and not state 0
            row = (tile - 1) // 10
            col = (tile - 1) % 10
            if row % 2 == 1:
                col = 9 - col
            board_grid_steady_state[9 - row, col] = steady_state_distribution[tile] # Access using 'tile' index


    sns.heatmap(board_grid_steady_state, cmap="viridis", linewidths=.5, cbar_kws={'label': 'Steady-State Probability'})
    plt.title(f"Steady-State Tile Occupation Probability Heatmap")
    plt.xticks([])
    plt.yticks([])
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()

def plot_win_prob_vs_ratio_separated_board_sizes(results_ns_fixed, results_nl_fixed, board_sizes, save_folder="plots/ratio_analysis", filename_prefix="win_prob_vs_ratio_board_sizes"):
    """
    Plots Win Probability vs Ns/Nl Ratio, with multiple lines for different board sizes on a single plot.
    Separated for Ns-fixed and Nl-fixed scenarios. Saves plot and data to CSV.
    """
    df_ns_fixed = pd.DataFrame(results_ns_fixed)
    df_nl_fixed = pd.DataFrame(results_nl_fixed)

    win_fractions = [0.5, 1/3, 0.25]
    fraction_labels = {'0.5': 'N/2', '0.3333333333333333': 'N/3', '0.25': 'N/4'}
    line_styles = ['-', '--', ':'] # Line styles for N/2, N/3, N/4 probs
    markers = ['o', '^', 's'] # Markers for N/2, N/3, N/4 probs
    colours = ['blue', 'purple', 'teal'] # Colours for N/2, N/3, N/4 probs


    # --- Plot for Ns Fixed ---
    fig = plt.figure(figsize=(12, 6))
    ax_ns = fig.add_subplot(1, 1, 1) # Single subplot


    for size in sorted(list(set(df_ns_fixed['board_size']))): # Iterate through board sizes
        df_subset_ns = df_ns_fixed[df_ns_fixed['board_size'] == size]
        for i, fraction in enumerate(win_fractions): # Iterate through win fractions
            prob_col_name = f'prob_win_{fraction_labels[str(fraction)]}_sim'.lower().replace('/', '').replace('.', '')
            ax_ns.plot(df_subset_ns['ratio'], df_subset_ns[prob_col_name],
                       marker=markers[i], linestyle=line_styles[i], color=colours[i],
                       label=f'{size}x{size} - N/{fraction_labels[str(fraction)]} Turns') # Label with board size and win condition


    ax_ns.set_xlabel("Ns/Nl Ratio")
    ax_ns.set_ylabel("Probability of Winning within N Turns")
    ax_ns.set_title("Win Probability vs Ns/Nl Ratio (Ns Fixed, by Board Size)")
    ax_ns.legend(loc='upper right', fontsize='small', ncol=2) # Adjusted legend
    ax_ns.grid(True)
    ax_ns.set_ylim(bottom=0, top=1.05) # Set y-axis limits
    fig.tight_layout()


    if save_folder:
        ns_fixed_folder = os.path.join(save_folder, "ns_fixed")
        os.makedirs(ns_fixed_folder, exist_ok=True)
        filepath_png = os.path.join(ns_fixed_folder, filename_prefix + "_ns_fixed.png")
        fig.savefig(filepath_png, bbox_inches='tight')
        print(f"Win prob vs ratio (Ns fixed, by board size) plot saved to: {filepath_png}")
    else:
        plt.show()
    plt.close(fig)



    # --- Plot for Nl Fixed ---
    fig = plt.figure(figsize=(12, 6))
    ax_nl = fig.add_subplot(1, 1, 1) # Single subplot


    for size in sorted(list(set(df_nl_fixed['board_size']))): # Iterate through board sizes
        df_subset_nl = df_nl_fixed[df_nl_fixed['board_size'] == size]
        for i, fraction in enumerate(win_fractions): # Iterate through win fractions
            prob_col_name = f'prob_win_{fraction_labels[str(fraction)]}_sim'.lower().replace('/', '').replace('.', '')
            ax_nl.plot(df_subset_nl['ratio'], df_subset_nl[prob_col_name],
                       marker=markers[i], linestyle=line_styles[i], color=colours[i],
                       label=f'{size}x{size} - N/{fraction_labels[str(fraction)]} Turns') # Label with board size and win condition


    ax_nl.set_xlabel("Ns/Nl Ratio")
    ax_nl.set_ylabel("Probability of Winning within N Turns")
    ax_nl.set_title("Win Probability vs Ns/Nl Ratio (Nl Fixed, by Board Size)")
    ax_nl.legend(loc='upper right', fontsize='small', ncol=2) # Adjusted legend
    ax_nl.grid(True)
    ax_nl.set_ylim(bottom=0, top=1.05) # Set y-axis limits
    fig.tight_layout()


    if save_folder:
        nl_fixed_folder = os.path.join(save_folder, "nl_fixed")
        os.makedirs(nl_fixed_folder, exist_ok=True)
        filepath_png = os.path.join(nl_fixed_folder, filename_prefix + "_nl_fixed.png")
        fig.savefig(filepath_png, bbox_inches='tight')
        print(f"Win prob vs ratio (Nl fixed, by board size) plot saved to: {filepath_png}")
    else:
        plt.show()
    plt.close(fig)

def plot_board_heatmap_representation_with_arrows(board_size, snakes, ladders, board_linear_size):
    """
    Plots a Snakes and Ladders board representation using sns.heatmap with labels AND arrows,
    dynamically sized.

    Args:
        board_size (int): Size of the board
        snakes (dict): Dictionary of snake positions
        ladders (dict): Dictionary of ladder positions
        board_linear_size (int): Linear dimension of the square board

    """
    cmap = sns.color_palette(['lightgray', 'red', 'green']) # Colour map

    grid_representation = create_labeled_board_representation(board_size, snakes, ladders, board_linear_size)
    grid_size = board_linear_size # Get grid size
    empty_board_data = np.zeros((grid_size, grid_size)) # Dynamic size for heatmap data

    fig, ax = plt.subplots(figsize=(12, 10))
    sns.heatmap(empty_board_data, annot=grid_representation, fmt='s', cmap=cmap,
                linewidths=1, linecolor='black', cbar=False, annot_kws={"fontsize": 8, "fontweight": "bold"}, ax=ax)

    ax.set_title(f"Snakes and Ladders Board ({board_size} Tiles) with Arrows - {board_linear_size}x{board_linear_size}", fontsize=14) # More descriptive title
    ax.set_xticks([])
    ax.set_yticks([])
    ax.invert_yaxis()

    # --- Add Arrows ---
    arrow_props_snake=dict(arrowstyle='-|>', mutation_scale=15, lw=1.5, color='red', connectionstyle="arc3,rad=0.") # Straight red arrows
    arrow_props_ladder=dict(arrowstyle='-|>', mutation_scale=15, lw=1.5, color='green', connectionstyle="arc3,rad=0.") # Straight green arrows


    for start_tile, end_tile in snakes.items():
        start_pos = _tile_to_position_dynamic_grid(start_tile, board_linear_size) # Use dynamic position function
        end_pos = _tile_to_position_dynamic_grid(end_tile, board_linear_size)     # Use dynamic position function
        arrow = FancyArrowPatch(start_pos, end_pos, transform=ax.transData, **arrow_props_snake,)
        ax.add_patch(arrow)

    for start_tile, end_tile in ladders.items():
        start_pos = _tile_to_position_dynamic_grid(start_tile, board_linear_size) # Use dynamic position function
        end_pos = _tile_to_position_dynamic_grid(end_tile, board_linear_size)     # Use dynamic position function
        arrow = FancyArrowPatch(start_pos, end_pos, transform=ax.transData, **arrow_props_ladder)
        ax.add_patch(arrow)
    # --- End Arrows ---

    return plt # Return plt object


def _tile_to_position_dynamic_grid(tile, board_linear_size):
    """Helper function to convert tile number to heatmap coordinates for dynamic grid."""
    grid_size = board_linear_size
    row = (tile - 1) // grid_size
    col = (tile - 1) % grid_size
    if row % 2 == 1:
        col = (grid_size - 1) - col
    return [col + 0.5, grid_size - 1 - row + 0.5] # Adjusted for dynamic grid size and inverted row

# %%
def run_simulations_and_markov(board_size, snakes, ladders, num_simulations, num_markov_simulations):
    """
    Encapsulates the simulation and Markov analysis to reduce repetition in the main loop.
    """
    turn_counts_simulation = []
    win_counts_n_turns_sim = {0.5: 0, 1/3: 0, 0.25: 0} # N/2, N/3, N/4 counters

    for _ in range(num_simulations):
        turns, _, _ = simulate_game(board_size, snakes, ladders)
        turn_counts_simulation.append(turns)
        for fraction, count in win_counts_n_turns_sim.items(): # Check for each fraction
            if turns <= board_size * fraction:
                win_counts_n_turns_sim[fraction] += 1


    avg_turns_simulation = sum(turn_counts_simulation) / num_simulations
    win_prob_n_turns_sim = {fraction: count / num_simulations for fraction, count in win_counts_n_turns_sim.items()} # Probabilities

    transition_matrix = create_transition_matrix(board_size, snakes, ladders)
    try:
        expected_turns_markov = calculate_expected_turns_markov(transition_matrix, board_size, snakes, ladders)
    except np.linalg.LinAlgError:
        expected_turns_markov = np.nan # Use NaN if Markov fails

    win_prob_n_turns_sim = {fraction: count / num_simulations for fraction, count in win_counts_n_turns_sim.items()} # Probabilities

    # --- Calculate Markov Win Probabilities ---
    win_probabilities_markov = calculate_markov_win_probabilities(transition_matrix, board_size)
    # --- End Markov Win Probabilities Calculation ---


    print("\n--- Win Probabilities (Debugging) ---") # Debugging print
    print("Win Probabilities (N/2 turns) (Sim):", win_prob_n_turns_sim[0.5])
    print("Win Probabilities (N/3 turns) (Sim):", win_prob_n_turns_sim[1/3])
    print("Win Probabilities (N/4 turns) (Sim):", win_prob_n_turns_sim[0.25])
    print("Win Probabilities (N/2 turns) (Markov):", win_probabilities_markov[0.5]) # Added Markov probs
    print("Win Probabilities (N/3 turns) (Markov):", win_probabilities_markov[1/3]) # Added Markov probs
    print("Win Probabilities (N/4 turns) (Markov):", win_probabilities_markov[0.25]) # Added Markov probs
    print("--- End Win Probabilities (Debugging) ---")


    transition_matrix = create_transition_matrix(board_size, snakes, ladders)

    return {
        'avg_turns_simulation': avg_turns_simulation,
        'expected_turns_markov': expected_turns_markov,
        'win_prob_n_turns_sim': win_prob_n_turns_sim,
        'win_prob_n_turns_markov': win_probabilities_markov, # Include Markov win probs in results
    }


# %%
def plot_avg_time_vs_ratio_individual(results, board_size, fixed_entity, save_folder="plots/ratio_analysis_individual"):
    """
    Plots Average Game Time vs Ns/Nl Ratio for a SINGLE board size and fixed entity type.
    Generates individual plots (not subplots) for better clarity.

    Args:
        results (dict):  Either results_ns_fixed or results_nl_fixed dictionary.
        board_size (int): The linear board size (e.g., 8, 10, 12...).
        fixed_entity (str): Either "Ns" or "Nl", indicating which entity is fixed.
        save_folder (str): Base folder to save plots.
    """

    df = pd.DataFrame(results)
    df_subset = df[df['board_size'] == board_size]

    fig, ax = plt.subplots(figsize=(8, 6)) # Create a new figure and axes for EACH plot

    # Plot Simulation data
    ax.plot(df_subset['ratio'], df_subset['avg_time_sim'], marker='o', linestyle='-', label='Simulation', color='blue' if fixed_entity == "Ns" else 'green', linewidth=2, markersize=8)

    # Plot Markov Model data
    ax.plot(df_subset['ratio'], df_subset['avg_time_markov'], marker='x', linestyle='--', label='Markov', color='blue' if fixed_entity == "Ns" else 'green', linewidth=2, markersize=8)

    ax.set_xlabel("Ns/Nl Ratio")
    ax.set_ylabel("Average Game Time (Turns)")
    ax.set_title(f"Avg Game Time vs Ns/Nl Ratio ({fixed_entity} Fixed, Board Size: {board_size}x{board_size})") # Clear title
    ax.legend(loc='best', fontsize='small')
    ax.grid(True)

    # --- Automatic Y-axis Scaling ---
    # Get the maximum and minimum data values for this subset.
    max_avg_time = df_subset[['avg_time_sim', 'avg_time_markov']].max().max()
    min_avg_time = df_subset[['avg_time_sim', 'avg_time_markov']].min().min()

    # Set y-axis limits with some padding:
    padding = (max_avg_time - min_avg_time) * 0.1  # 10% padding
    ax.set_ylim(bottom=max(0, min_avg_time - padding), top=max_avg_time + padding)

    fig.tight_layout()


    if save_folder:
        entity_folder = os.path.join(save_folder, f"{fixed_entity.lower()}_fixed") # ns_fixed or nl_fixed
        size_folder = os.path.join(entity_folder, f"board_{board_size}x{board_size}")
        os.makedirs(size_folder, exist_ok=True)
        filepath_png = os.path.join(size_folder, f"avg_time_vs_ratio_{board_size}x{board_size}_{fixed_entity}_fixed.png")
        fig.savefig(filepath_png, bbox_inches='tight')
        print(f"Average game time vs ratio plot ({board_size}x{board_size}, {fixed_entity} Fixed) saved to: {filepath_png}")

        # --- Save raw data to CSV (Optional - for individual plots) ---
        # You *could* also save the data for this specific plot to a CSV, if you want
        # very granular data files.  Uncomment the following if needed.
        # filepath_csv = os.path.join(size_folder, f"avg_time_vs_ratio_{board_size}x{board_size}_{fixed_entity}_fixed.csv")
        # df_subset.to_csv(filepath_csv, index=False)
        # print(f"Data saved to: {filepath_csv}")


    else:
        plt.show()

    plt.close(fig)

# %%
def run_analysis(board_size=100, num_snakes=10, num_ladders=10, num_simulations=10000, num_markov_simulations=10000, display_labeled_board=False, board_linear_size=10, save_plots=True): # Added save_plots flag
    """
    Runs the complete analysis of the Snakes and Ladders game with optional labeled board display, saving plots.
    """

    turn_counts_simulation = []
    all_positions_visited = []
    all_entity_triggers = []

    snakes, ladders = create_snakes_and_ladders(board_size, num_snakes, num_ladders)

    print("\nBoard Configuration:")
    print("Snakes:", {k: v for k, v in snakes.items()})
    print("Ladders:", {k: v for k, v in ladders.items()})

    if display_labeled_board:
        save_board_folder = None # Don't save individual board layout if just displaying
        if save_plots:
            save_board_folder = "plots/board_layouts" # Save board layouts in this folder if saving is enabled
        plot_board_heatmap_representation_with_arrows(board_size, snakes, ladders, board_linear_size, save_folder=save_board_folder, filename=f"board_layout_{board_size}tiles_{num_snakes}s_{num_ladders}ladders.png") # Save board layout

    for _ in range(num_simulations):
        turns, positions, triggers = simulate_game(board_size, snakes, ladders,
                                                 record_positions=True,
                                                 record_entity_triggers=True)
        turn_counts_simulation.append(turns)
        all_positions_visited.append(positions)
        all_entity_triggers.append(triggers)

    avg_turns_simulation = sum(turn_counts_simulation) / num_simulations
    print(f"\nSimulation Results:")
    print(f"Average Turns (Simulation): {avg_turns_simulation:.2f}")
    print(f"Min Turns: {min(turn_counts_simulation)}")
    print(f"Max Turns: {max(turn_counts_simulation)}")

    transition_matrix = create_transition_matrix(board_size, snakes, ladders)
    try:
        expected_turns_markov = calculate_expected_turns_markov(transition_matrix, board_size, snakes, ladders)
        print(f"\nMarkov Analysis:")
        print(f"Expected Turns (Markov Model): {expected_turns_markov:.2f}")
    except np.linalg.LinAlgError:
        print("Markov analysis failed due to singular matrix")
        expected_turns_markov = None

    steady_state_distribution = calculate_steady_state_distribution(transition_matrix, board_size, snakes, ladders)

    # Simulate games from Markov model distribution
    turn_counts_markov = simulate_game_markov_distribution(transition_matrix, num_markov_simulations, board_size)

    if save_plots: # Save plots if save_plots is True
        base_folder = "plots/single_board_analysis" # Base folder for single board analysis
        board_config_folder = os.path.join(base_folder, f"board_{board_size}tiles_{num_snakes}s_{num_ladders}ladders") # Subfolder for board config
        plot_turn_distribution(turn_counts_simulation, turn_counts_markov, num_snakes, num_ladders, num_simulations, save_folder=board_config_folder, filename="turn_distribution.png")
        plot_board_heatmap(all_positions_visited, board_size, snakes, ladders, num_simulations, save_folder=board_config_folder, filename="visit_heatmap.png")
        plot_entity_trigger_heatmap(all_entity_triggers, board_size, snakes, ladders, num_simulations, save_folder=board_config_folder, filename="entity_trigger_heatmap.png")
        plot_steady_state_heatmap(steady_state_distribution, board_size, snakes, ladders, save_folder=board_config_folder, filename="steady_state_heatmap.png")
    else: # Show plots if not saving
        plot_turn_distribution(turn_counts_simulation, turn_counts_markov, num_snakes, num_ladders, num_simulations)
        plot_board_heatmap(all_positions_visited, board_size, snakes, ladders, num_simulations)
        plot_entity_trigger_heatmap(all_entity_triggers, board_size, snakes, ladders, num_simulations)
        plot_steady_state_heatmap(steady_state_distribution, board_size, snakes, ladders)



def run_board_scaling_analysis(board_sizes, entity_density=0.1, num_ratios=4, num_simulations=1000, num_markov_simulations=1000, display_board=False, save_plots=True):
    """
    Runs analysis, calls individual and faceted plotting functions.
    """

    ratio_increments = np.linspace(0.5, 2.0, num_ratios)

    # Data storage (remains same)
    results_ns_fixed = {'board_size': [], 'ratio': [], 'avg_time_sim': [], 'avg_time_markov': [],
                            'prob_win_n2_sim': [], 'prob_win_n3_sim': [], 'prob_win_n4_sim': [],
                            'prob_win_n2_markov': [], 'prob_win_n3_markov': [], 'prob_win_n4_markov': []} # Added Markov win probs storage
    results_nl_fixed = {'board_size': [], 'ratio': [], 'avg_time_sim': [], 'avg_time_markov': [],
                            'prob_win_n2_sim': [], 'prob_win_n3_sim': [], 'prob_win_n4_sim': [],
                            'prob_win_n2_markov': [], 'prob_win_n3_markov': [], 'prob_win_n4_markov': []} # Added Markov win probs storage


    for board_size_linear in board_sizes:
      # ... (rest of the board size and ratio loop code - no changes within the loops) ...
        # --- Operation Set 1: Ns Fixed by Density, Nl by Ratio ---
        board_area = board_size_linear**2 # board_area is defined here
        board_size = board_area # Use board_area for board_size
        num_snakes_fixed = int(entity_density * board_area)
        print(f"\n--- Board Size: {board_size_linear}x{board_size_linear} ({board_area} tiles), Ns Fixed = {num_snakes_fixed} ---")

        for ratio in ratio_increments:
            num_ladders = max(1, int(num_snakes_fixed / ratio))
            print(f"\n  Ratio (Ns/Nl): {ratio:.1f}, Nl: {num_ladders}")

            snakes, ladders = create_snakes_and_ladders(board_size, num_snakes_fixed, num_ladders)

            if display_board:
                save_board_folder = None
                if save_plots:
                    save_board_folder = os.path.join("plots/board_layouts", "board_scaling", "ns_fixed", f"board_{board_size_linear}x{board_size_linear}", f"ratio_{ratio:.1f}")
                board_plot = plot_board_heatmap_representation_with_arrows(board_size, snakes, ladders, board_size_linear, save_folder=save_board_folder, filename="board_layout.png")
                if board_plot:
                    board_plot.suptitle(f"Ns Fixed, Ratio {ratio:.1f}, Board {board_size_linear}x{board_size_linear}")

            sim_results = run_simulations_and_markov(board_size, snakes, ladders, num_simulations, num_markov_simulations)

            # Store results
            results_ns_fixed['board_size'].append(board_size_linear)
            results_ns_fixed['ratio'].append(ratio)
            results_ns_fixed['avg_time_sim'].append(sim_results['avg_turns_simulation'])
            results_ns_fixed['avg_time_markov'].append(sim_results['expected_turns_markov'])
            results_ns_fixed['prob_win_n2_sim'].append(sim_results['win_prob_n_turns_sim'][0.5])
            results_ns_fixed['prob_win_n3_sim'].append(sim_results['win_prob_n_turns_sim'][1/3])
            results_ns_fixed['prob_win_n4_sim'].append(sim_results['win_prob_n_turns_sim'][0.25])
            results_ns_fixed['prob_win_n2_markov'].append(sim_results['win_prob_n_turns_markov'][0.5]) # Store Markov win probs
            results_ns_fixed['prob_win_n3_markov'].append(sim_results['win_prob_n_turns_markov'][1/3]) # Store Markov win probs
            results_ns_fixed['prob_win_n4_markov'].append(sim_results['win_prob_n_turns_markov'][0.25]) # Store Markov win probs


        # --- Operation Set 2: Nl Fixed by Density, Ns by Ratio ---
        num_ladders_fixed = int(entity_density * board_area)
        print(f"\n--- Board Size: {board_size_linear}x{board_size_linear} ({board_area} tiles), Nl Fixed = {num_ladders_fixed} ---")

        for ratio in ratio_increments:
            num_snakes = max(1, int(num_ladders_fixed * ratio))
            print(f"\n  Ratio (Ns/Nl): {ratio:.1f}, Ns: {num_snakes}")

            snakes, ladders = create_snakes_and_ladders(board_size, num_snakes, num_ladders_fixed)

            if display_board:
                save_board_folder = None
                if save_plots:
                    save_board_folder = os.path.join("plots/board_layouts", "board_scaling", "nl_fixed", f"board_{board_size_linear}x{board_size_linear}", f"ratio_{ratio:.1f}")
                board_plot = plot_board_heatmap_representation_with_arrows(board_size, snakes, ladders, board_size_linear, save_folder=save_board_folder, filename="board_layout.png")
                if board_plot:
                    board_plot.suptitle(f"Nl Fixed, Ratio {ratio:.1f}, Board {board_size_linear}x{board_size_linear}")

            sim_results = run_simulations_and_markov(board_size, snakes, ladders, num_simulations, num_markov_simulations)

            # Store results
            results_nl_fixed['board_size'].append(board_size_linear)
            results_nl_fixed['ratio'].append(ratio)
            results_nl_fixed['avg_time_sim'].append(sim_results['avg_turns_simulation'])
            results_nl_fixed['avg_time_markov'].append(sim_results['expected_turns_markov'])
            results_nl_fixed['prob_win_n2_sim'].append(sim_results['win_prob_n_turns_sim'][0.5])
            results_nl_fixed['prob_win_n3_sim'].append(sim_results['win_prob_n_turns_sim'][1/3])
            results_nl_fixed['prob_win_n4_sim'].append(sim_results['win_prob_n_turns_sim'][0.25])
            results_nl_fixed['prob_win_n2_markov'].append(sim_results['win_prob_n_turns_markov'][0.5]) # Store Markov win probs
            results_nl_fixed['prob_win_n3_markov'].append(sim_results['win_prob_n_turns_markov'][1/3]) # Store Markov win probs
            results_nl_fixed['prob_win_n4_markov'].append(sim_results['win_prob_n_turns_markov'][0.25]) # Store Markov win probs


    # --- Create DataFrames AFTER the loops have completed ---
    df_ns_fixed = pd.DataFrame(results_ns_fixed)
    df_nl_fixed = pd.DataFrame(results_nl_fixed)

    # --- Create Plots with Saving ---
    if save_plots:
        plot_folder_board_size = os.path.join("plots", "board_scaling")
        plot_avg_time_vs_board_size_separated(results_ns_fixed, results_nl_fixed, save_folder=plot_folder_board_size, filename_prefix="avg_time_vs_board_size")
        plot_win_prob_vs_board_size_separated(results_ns_fixed, results_nl_fixed, save_folder=plot_folder_board_size, filename_prefix="win_prob_vs_board_size")

        # Individual ratio plots:
        plot_folder_ratio_individual = os.path.join("plots", "ratio_analysis_individual")
        for size in board_sizes:
            plot_avg_time_vs_ratio_individual(results_ns_fixed, size, "Ns", save_folder=plot_folder_ratio_individual) # Ns fixed plots
            plot_avg_time_vs_ratio_individual(results_nl_fixed, size, "Nl", save_folder=plot_folder_ratio_individual) # Nl fixed plots

        # Faceted ratio plots:
        plot_folder_ratio = os.path.join("plots", "ratio_analysis")
        plot_avg_time_vs_ratio_faceted(results_ns_fixed, results_nl_fixed, board_sizes, save_folder=plot_folder_ratio, filename="avg_time_vs_ratio_faceted.png") # Keep avg time faceted plot
        plot_win_prob_vs_ratio_faceted(results_ns_fixed, results_nl_fixed, board_sizes, save_folder=plot_folder_ratio, filename="win_prob_vs_ratio_faceted.png") # Re-introduce faceted win prob plot
        plot_win_prob_vs_ratio_separated_board_sizes(results_ns_fixed, results_nl_fixed, board_sizes, save_folder=plot_folder_ratio, filename_prefix="win_prob_vs_ratio_board_sizes") # Keep multiple lines plot

        # --- Save raw data to CSV in base 'plots' folder ---
        df_ns_fixed.to_csv("plots/results_ns_fixed.csv", index=False) # Now df_ns_fixed is defined here
        df_nl_fixed.to_csv("plots/results_nl_fixed.csv", index=False) # Now df_nl_fixed is defined here
        print("Raw data (Ns fixed) saved to: plots/results_ns_fixed.csv")
        print("Raw data (Nl fixed) saved to: plots/results_nl_fixed.csv")

    else: # Otherwise just show plots
        for size in board_sizes:
              plot_avg_time_vs_ratio_individual(results_ns_fixed, size, "Ns")  # Ns fixed
              plot_avg_time_vs_ratio_individual(results_nl_fixed, size, "Nl")  # Nl fixed
        plot_win_prob_vs_ratio_separated_board_sizes(results_ns_fixed, results_nl_fixed, board_sizes) # Keep multiple lines plot
        plot_avg_time_vs_board_size_separated(results_ns_fixed, results_nl_fixed)
        plot_win_prob_vs_board_size_separated(results_ns_fixed, results_nl_fixed)
        plot_win_prob_vs_ratio_faceted(results_ns_fixed, results_nl_fixed, board_sizes) # Re-introduce faceted win prob plot

    return results_ns_fixed, results_nl_fixed

# %%
if __name__ == "__main__":
    board_linear_sizes = range(8, 22, 2)
    results_ns_fixed, results_nl_fixed = run_board_scaling_analysis(
        board_linear_sizes,
        entity_density=0.1,
        num_ratios=4,
        num_simulations=1000, # Reduced for testing, increase for full run
        num_markov_simulations=1000, # Reduced for testing, increase for full run
        display_board=False, # Keep False for full runs, True to check board layouts
        save_plots=True # Enable saving plots
    )

    print("\n--- Analysis Complete - Plots and Raw Data Saved to 'plots' folder ---")


--- Board Size: 8x8 (64 tiles), Ns Fixed = 6 ---

  Ratio (Ns/Nl): 0.5, Nl: 12

--- Board Generation (Debugging) ---
Number of snakes placed: 6
Number of ladders placed: 12
Snakes: {26: 4, 23: 1, 45: 13, 34: 2, 55: 32, 43: 21}
Ladders: {3: 52, 30: 51, 33: 46, 22: 36, 12: 53, 27: 58, 24: 63, 18: 44, 20: 47, 16: 49, 15: 60, 29: 48}
--- End Board Generation (Debugging) ---
Valid board generated after 1 regeneration attempts.

--- Win Probabilities (Debugging) ---
Win Probabilities (N/2 turns) (Sim): 0.963
Win Probabilities (N/3 turns) (Sim): 0.838
Win Probabilities (N/4 turns) (Sim): 0.698
Win Probabilities (N/2 turns) (Markov): 0.9740139769084355
Win Probabilities (N/3 turns) (Markov): 0.8598930514630342
Win Probabilities (N/4 turns) (Markov): 0.7172564502829667
--- End Win Probabilities (Debugging) ---

  Ratio (Ns/Nl): 1.0, Nl: 6

--- Board Generation (Debugging) ---
Number of snakes placed: 6
Number of ladders placed: 6
Snakes: {63: 16, 46: 4, 33: 9, 54: 20, 37: 12, 25: 1}
Ladders: {